<a href="https://colab.research.google.com/github/Kksahoo02/Code-Energy-Prediction/blob/main/STATIC_FEATURES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# PURE STATIC FEATURES - Final Debugged Script
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import IsolationForest, RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.feature_selection import VarianceThreshold
from lightgbm import LGBMRegressor, log_evaluation

# ==========================
# CONFIG
# ==========================
DATASET_PATH = '/content/ml_code_dataset.csv'  # <- update to your file
TARGET = 'energy_consumption_joules'

# Lists (static/dynamic/metadata)
DYNAMIC_FEATURES = [
    'execution_time', 'runtime_seconds', 'elapsed_time', 'duration',
    'memory_peak_mb', 'memory_avg_mb', 'memory_usage', 'memory_allocated',
    'memory_reserved', 'memory_max', 'ram_usage', 'heap_size',
    'cpu_percent_avg', 'cpu_percent_peak', 'cpu_usage', 'cpu_time',
    'user_time', 'system_time', 'cpu_cycles',
    'disk_read_mb', 'disk_write_mb', 'io_read', 'io_write',
    'disk_operations', 'read_bytes', 'write_bytes',
    'co2_emissions_g', 'power_draw_watts', 'energy_pkg_joules',
    'energy_ram_joules', 'energy_consumption', 'carbon_footprint',
    'gpu_used', 'gpu_memory', 'gpu_utilization', 'cuda_memory',
    'network_sent_mb', 'network_recv_mb', 'bytes_sent', 'bytes_received',
    'packets_sent', 'packets_received',
    'process_count', 'thread_count', 'context_switches', 'num_threads',
    'num_processes', 'child_processes',
    'cache_misses', 'cache_hits', 'instructions_retired', 'cycles_elapsed',
    'branch_misses', 'page_faults', 'tlb_misses',
    'runtime_errors', 'memory_errors', 'exception_count', 'error_count',
    'warning_count', 'num_exceptions',
    'output_size', 'return_value', 'stdout_length', 'stderr_length',
    'output_lines', 'print_statements_executed',
    'system_load', 'available_memory', 'free_disk', 'temperature',
]

METADATA_COLS = [
    'file_path', 'file_name', 'repo_name', 'repo_url', 'timestamp',
    'executed_successfully', 'syntax_valid', 'run_id', 'index', 'id',
    'file_id', 'repository', 'author', 'date_created', 'date_modified'
]

dynamic_keywords = [
    'time', 'memory', 'cpu', 'disk', 'io', 'network', 'gpu',
    'peak', 'avg', 'max', 'min', 'usage', 'consumed', 'allocated',
    'runtime', 'execution', 'process', 'thread', 'energy', 'power',
    'emission', 'carbon', 'watt', 'joule', 'exception', 'error',
    'output', 'result', 'performance', 'utilization', 'load'
]

# Helper for safe column printing
def safe_print_cols(cols, title=None, max_show=40):
    if title:
        print(title)
    for c in (cols[:max_show]):
        print(f"   - {c}")
    if len(cols) > max_show:
        print(f"   ... and {len(cols)-max_show} more")

# ==========================
# LOAD & BASIC FILTERING
# ==========================
print("=" * 80)
print("PURE STATIC FEATURES MODEL - DEBUGGED")
print("=" * 80)

df = pd.read_csv(DATASET_PATH)
print(f"\n📊 Initial dataset: {len(df):,} rows × {df.shape[1]} columns")

# Filter for executed_successfully if available
if 'executed_successfully' in df.columns:
    before = len(df)
    df = df[df['executed_successfully'] == True].copy()
    after = len(df)
    print(f"✅ After filtering executed_successfully: {after:,} rows ({after/before*100:.1f}% of pre-filter)")

# Ensure target exists and remove zero/NaN
if TARGET not in df.columns:
    raise SystemExit(f"ERROR: Target '{TARGET}' not found. Columns: {df.columns.tolist()}")

df = df.dropna(subset=[TARGET])
df = df[df[TARGET] > 0].copy()
print(f"✅ After removing zero/NaN target: {len(df):,} rows")

if len(df) < 10:
    raise SystemExit("ERROR: Too few rows remain after initial filtering to train a model safely.")

# ==========================
# IDENTIFY PURE STATIC FEATURES
# ==========================
print("\n🔍 Identifying PURE STATIC features...")

all_cols = df.columns.tolist()
candidates = [c for c in all_cols if c not in (DYNAMIC_FEATURES + METADATA_COLS + [TARGET])]

static_features = []
removed_suspicious = []
for col in candidates:
    low = col.lower()
    if any(k in low for k in dynamic_keywords):
        removed_suspicious.append(col)
    else:
        static_features.append(col)

print(f"\n📋 Feature filtering summary:")
print(f"   Total columns: {len(all_cols)}")
print(f"   Removed metadata columns: {len([c for c in all_cols if c in METADATA_COLS])}")
print(f"   Removed explicit dynamic features: {len([c for c in all_cols if c in DYNAMIC_FEATURES])}")
print(f"   Removed suspicious keyword matches: {len(removed_suspicious)}")
print(f"   ✅ Candidate static features (pre-numeric): {len(static_features)}")

if removed_suspicious:
    safe_print_cols(removed_suspicious, title="⚠️ Example removed suspicious columns:", max_show=20)

if len(static_features) == 0:
    print("\n❌ No static features detected. Listing all columns for debugging:")
    safe_print_cols(all_cols, title="All columns:")
    raise SystemExit("No static features found. Adjust lists or dataset.")

# ==========================
# KEEP ONLY NUMERIC STATIC FEATURES
# ==========================
X_raw = df[static_features].copy()
y = df[TARGET].copy()

# Coerce non-numeric to numeric where possible (non-convertible => NaN)
for col in X_raw.columns:
    if not pd.api.types.is_numeric_dtype(X_raw[col]):
        X_raw[col] = pd.to_numeric(X_raw[col], errors='coerce')

# Drop columns that became all-NaN
num_cols = [c for c in X_raw.columns if X_raw[c].notna().any()]
X = X_raw[num_cols].copy()
dropped_non_numeric = set(static_features) - set(num_cols)
if dropped_non_numeric:
    print(f"\nℹ️ Dropped non-numeric/all-NaN static features: {len(dropped_non_numeric)}")
    safe_print_cols(sorted(list(dropped_non_numeric)), title=" Examples dropped:", max_show=20)

print(f"\n📐 Feature matrix: {X.shape[0]:,} rows × {X.shape[1]} features")

if X.shape[1] == 0:
    raise SystemExit("ERROR: No numeric static features available after coercion.")

# ==========================
# CLEAN INF / MISSING
# ==========================
print("\n🧹 Cleaning feature matrix (infs -> NaN, drop high-missing, fill medians)...")
X.replace([np.inf, -np.inf], np.nan, inplace=True)

missing_pct = X.isna().sum() / len(X)
cols_to_drop = missing_pct[missing_pct > 0.5].index.tolist()
if cols_to_drop:
    X.drop(columns=cols_to_drop, inplace=True)
    print(f"   Dropped {len(cols_to_drop)} columns with >50% missing")

# Fill remaining NaNs with median
for col in X.columns:
    if X[col].isna().sum() > 0:
        X[col].fillna(X[col].median(), inplace=True)

print(f"✅ After cleaning: {X.shape[1]} features remain")

# ==========================
# REMOVE CONSTANT / LOW VARIANCE
# ==========================
print("\n🔧 Removing constant and very low-variance features...")
constant_cols = [col for col in X.columns if X[col].nunique() <= 1]
if constant_cols:
    X.drop(columns=constant_cols, inplace=True)
    print(f"   Removed {len(constant_cols)} constant features")

if X.shape[1] > 0:
    vt = VarianceThreshold(threshold=0.0)
    vt.fit(X)
    variances = vt.variances_
    variance_cutoff = np.percentile(variances, 5)
    keep_mask = variances > variance_cutoff
    cols_kept = X.columns[keep_mask]
    X = X[cols_kept]
    print(f"   After low-variance filtering: {X.shape[1]} features remain")

if X.shape[1] == 0:
    raise SystemExit("ERROR: No features remain after variance filtering.")

print(f"✅ Final static feature count: {X.shape[1]}")

# ==========================
# AGGRESSIVE OUTLIER REMOVAL
# ==========================
print("\n🔍 Aggressive outlier removal (IsolationForest -> quantiles -> z-score)...")
X_clean = X.copy()
y_clean = y.loc[X_clean.index].copy()

print(f"   Initial samples: {X_clean.shape[0]:,}")

# IsolationForest (skip if too few samples)
if X_clean.shape[0] > 20:
    try:
        iso = IsolationForest(contamination=0.10, random_state=42, n_estimators=200, max_samples='auto')
        pred = iso.fit_predict(X_clean)
        mask_iso = pred == 1
        X_clean = X_clean.loc[mask_iso].copy()
        y_clean = y_clean.loc[mask_iso].copy()
        print(f"   After IsolationForest (10% removal): {X_clean.shape[0]:,} samples")
    except Exception as e:
        print(f"   ⚠️ IsolationForest failed: {e}. Skipping this step.")
else:
    print("   ⚠️ Too few samples for IsolationForest - skipping.")

# Quantile filter on target
q_low = y_clean.quantile(0.01)
q_high = y_clean.quantile(0.97)
mask_q = (y_clean >= q_low) & (y_clean <= q_high)
X_clean = X_clean.loc[mask_q].copy()
y_clean = y_clean.loc[mask_q].copy()
print(f"   After quantile filtering (1%-97%): {X_clean.shape[0]:,} samples")

# Z-score filter per feature
stds = X_clean.std()
zero_std_cols = stds[stds == 0].index.tolist()
if zero_std_cols:
    X_clean.drop(columns=zero_std_cols, inplace=True)
    print(f"   Dropped {len(zero_std_cols)} zero-std columns before z-score filtering")

if X_clean.shape[0] > 0 and X_clean.shape[1] > 0:
    z_scores = np.abs((X_clean - X_clean.mean()) / X_clean.std(ddof=0))
    mask_z = (z_scores < 3.5).all(axis=1)
    X_clean = X_clean.loc[mask_z].copy()
    y_clean = y_clean.loc[mask_z].copy()
    print(f"   After z-score filtering (3.5σ): {X_clean.shape[0]:,} samples")
else:
    print("   ⚠️ No samples/features left to perform z-score filtering.")

removed_count = X.shape[0] - X_clean.shape[0]
print(f"✅ Total outliers removed (feature-rows): {removed_count:,} ({(removed_count / max(1, X.shape[0]))*100:.1f}%)")

if X_clean.shape[0] < 20:
    print("⚠️ Warning: Very few samples remain after outlier removal. Model may be unstable.")

# ==========================
# TARGET TRANSFORM
# ==========================
y_log = np.log1p(y_clean)
print("\n📈 Applied log1p to target:")
print(f"   Original target range: [{y_clean.min():.2f}, {y_clean.max():.2f}]")
print(f"   Log-transformed range: [{y_log.min():.2f}, {y_log.max():.2f}]")

# ==========================
# SCALING
# ==========================
scaler = RobustScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X_clean), columns=X_clean.columns, index=X_clean.index)
print("✅ RobustScaler applied")

# ==========================
# TRAIN-TEST SPLIT
# ==========================
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_log, test_size=0.2, random_state=42)
print(f"\n📊 Train/Test split: train={X_train.shape[0]:,}, test={X_test.shape[0]:,}")

# ==========================
# LIGHTGBM TRAINING (no verbose arg)
# ==========================
print(f"\n🚀 Training LightGBM (features: {X_train.shape[1]})")
lgbm = LGBMRegressor(
    objective='regression',
    n_estimators=2000,
    learning_rate=0.01,
    num_leaves=31,
    max_depth=8,
    min_child_samples=20,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=0.1,
    random_state=42,
    n_jobs=-1,
    verbose=-1
)

# Use callback for periodic logging (optional). Remove callbacks parameter for silent run.
lgbm.fit(
    X_train,
    y_train,
    eval_set=[(X_test, y_test)],
    eval_metric='rmse',
    callbacks=[log_evaluation(100)]
)

# ==========================
# LGBM EVALUATION
# ==========================
print("\n" + "=" * 80)
print("LIGHTGBM PERFORMANCE")
print("=" * 80)

y_train_pred_log = lgbm.predict(X_train)
y_test_pred_log = lgbm.predict(X_test)

r2_train_log = r2_score(y_train, y_train_pred_log)
r2_test_log = r2_score(y_test, y_test_pred_log)
print("\n📊 Log-scale R²:")
print(f"   Train R²: {r2_train_log:.6f}")
print(f"   Test  R²: {r2_test_log:.6f}")

# Back to original scale
y_train_actual = np.expm1(y_train)
y_train_pred = np.expm1(y_train_pred_log)
y_test_actual = np.expm1(y_test)
y_test_pred = np.expm1(y_test_pred_log)

r2_train_orig = r2_score(y_train_actual, y_train_pred)
r2_test_orig = r2_score(y_test_actual, y_test_pred)
mae_train = mean_absolute_error(y_train_actual, y_train_pred)
mae_test = mean_absolute_error(y_test_actual, y_test_pred)
rmse_train = np.sqrt(mean_squared_error(y_train_actual, y_train_pred))
rmse_test = np.sqrt(mean_squared_error(y_test_actual, y_test_pred))

print("\n⚡ Original scale performance:")
print(f"   Train R²: {r2_train_orig:.6f}")
print(f"   Test  R²: {r2_test_orig:.6f}")

print("\n📏 Error metrics (Joules):")
print(f"   Train MAE: {mae_train:,.2f} J")
print(f"   Test  MAE: {mae_test:,.2f} J")
print(f"   Train RMSE: {rmse_train:,.2f} J")
print(f"   Test  RMSE: {rmse_test:,.2f} J")

mean_energy = y_test_actual.mean() if len(y_test_actual) else np.nan
if not np.isnan(mean_energy) and mean_energy != 0:
    print(f"   Mean energy (test): {mean_energy:,.2f} J")
    print(f"   MAE as % of mean: {(mae_test/mean_energy)*100:.2f}%")

# Feature importance
fi = pd.DataFrame({'feature': X_train.columns, 'importance': lgbm.feature_importances_}).sort_values('importance', ascending=False)
print("\n🎯 Top LightGBM features:")
for idx, row in fi.head(20).iterrows():
    print(f"   {row['feature']:40s} {row['importance']:8.1f}")

# ==========================
# LGBM CROSS-VALIDATION
# ==========================
print("\n🔄 5-Fold CV (LGBM, log-scale):")
try:
    cv_scores = cross_val_score(lgbm, X_scaled, y_log, cv=5, scoring='r2', n_jobs=-1)
    print(f"   CV R²: {[f'{s:.4f}' for s in cv_scores]}")
    print(f"   Mean CV R²: {cv_scores.mean():.6f} ±{cv_scores.std():.6f}")
except Exception as e:
    print(f"   ⚠️ CV failed: {e}")

# ==========================
# RANDOM FOREST (max_features fixed)
# ==========================
print("\n🌲 Training RandomForestRegressor (max_features='sqrt')")

rf = RandomForestRegressor(
    n_estimators=600,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='sqrt',  # fixed from 'auto'
    bootstrap=True,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)

# RF evaluation
print("\n" + "=" * 80)
print("RANDOM FOREST PERFORMANCE")
print("=" * 80)

rf_train_log_pred = rf.predict(X_train)
rf_test_log_pred = rf.predict(X_test)

rf_r2_train_log = r2_score(y_train, rf_train_log_pred)
rf_r2_test_log = r2_score(y_test, rf_test_log_pred)
print("\n📊 Log-scale R² (RF):")
print(f"   Train R²: {rf_r2_train_log:.6f}")
print(f"   Test  R²: {rf_r2_test_log:.6f}")

rf_train_actual = np.expm1(y_train)
rf_train_pred = np.expm1(rf_train_log_pred)
rf_test_actual = np.expm1(y_test)
rf_test_pred = np.expm1(rf_test_log_pred)

rf_r2_train_orig = r2_score(rf_train_actual, rf_train_pred)
rf_r2_test_orig = r2_score(rf_test_actual, rf_test_pred)

rf_mae_train = mean_absolute_error(rf_train_actual, rf_train_pred)
rf_mae_test = mean_absolute_error(rf_test_actual, rf_test_pred)
rf_rmse_train = np.sqrt(mean_squared_error(rf_train_actual, rf_train_pred))
rf_rmse_test = np.sqrt(mean_squared_error(rf_test_actual, rf_test_pred))

print("\n⚡ Original-scale (RF):")
print(f"   Train R²: {rf_r2_train_orig:.6f}")
print(f"   Test  R²: {rf_r2_test_orig:.6f}")

print("\n📏 Error metrics (RF, Joules):")
print(f"   Train MAE: {rf_mae_train:,.2f} J")
print(f"   Test  MAE: {rf_mae_test:,.2f} J")
print(f"   Train RMSE: {rf_rmse_train:,.2f} J")
print(f"   Test  RMSE: {rf_rmse_test:,.2f} J")

if not np.isnan(mean_energy) and mean_energy != 0:
    print(f"   RF MAE as % of mean energy: {(rf_mae_test/mean_energy)*100:.2f}%")

# RF cross-val
print("\n🔄 5-Fold CV (RF, log-scale):")
try:
    rf_cv_scores = cross_val_score(rf, X_scaled, y_log, cv=5, scoring='r2', n_jobs=-1)
    print(f"   CV R²: {[f'{s:.4f}' for s in rf_cv_scores]}")
    print(f"   Mean CV R²: {rf_cv_scores.mean():.6f} ±{rf_cv_scores.std():.6f}")
except Exception as e:
    print(f"   ⚠️ RF CV failed: {e}")

# ==========================
# FINAL SUMMARY
# ==========================
print("\n" + "=" * 80)
print("🎉 FINAL SUMMARY")
print("=" * 80)
print(f"✅ Trained on {X_clean.shape[0]:,} clean samples")
print(f"✅ Using {X_clean.shape[1]} pure static features")
print(f"✅ Outliers removed (by features/rows): {removed_count:,}")
try:
    print(f"✅ LightGBM Test R² (log): {r2_test_log:.6f}")
    print(f"✅ LightGBM Test MAE (J): {mae_test:,.2f}")
except:
    pass
try:
    print(f"✅ RF Test R² (orig): {rf_r2_test_orig:.6f}")
except:
    pass
print("=" * 80)

print("\n📋 Final static features used:")
safe_print_cols(sorted(X.columns), max_show=200)


PURE STATIC FEATURES MODEL - DEBUGGED

📊 Initial dataset: 6,711 rows × 107 columns
✅ After filtering executed_successfully: 1,567 rows (23.3% of pre-filter)
✅ After removing zero/NaN target: 1,567 rows

🔍 Identifying PURE STATIC features...

📋 Feature filtering summary:
   Total columns: 107
   Removed metadata columns: 4
   Removed explicit dynamic features: 4
   Removed suspicious keyword matches: 34
   ✅ Candidate static features (pre-numeric): 64
⚠️ Example removed suspicious columns:
   - file_extension
   - avg_line_length
   - max_line_length
   - min_line_length
   - comment_ratio
   - unique_indentation_levels
   - avg_indentation
   - max_indentation
   - std_indentation
   - functions_count
   - async_functions_count
   - lambda_functions
   - list_comprehensions
   - dict_comprehensions
   - set_comprehensions
   - generator_expressions
   - binary_operations
   - unary_operations
   - boolean_operations
   - max_nesting_depth
   ... and 14 more

📐 Feature matrix: 1,567 row

In [ ]:
# @title
# STATIC + DYNAMIC
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import RobustScaler, PowerTransformer
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from lightgbm import LGBMRegressor
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION
# ============================================================================
DATASET_PATH = '/content/ml_code_dataset.csv'

# Comprehensive dynamic features list
DYNAMIC_FEATURES = [
    'execution_time', 'runtime_seconds', 'elapsed_time', 'duration',
    'memory_peak_mb', 'memory_avg_mb', 'memory_usage', 'memory_allocated',
    'memory_reserved', 'memory_max', 'ram_usage', 'heap_size',
    'cpu_percent_avg', 'cpu_percent_peak', 'cpu_usage', 'cpu_time',
    'user_time', 'system_time', 'cpu_cycles',
    'disk_read_mb', 'disk_write_mb', 'io_read', 'io_write',
    'disk_operations', 'read_bytes', 'write_bytes',
    'co2_emissions_g', 'power_draw_watts', 'energy_pkg_joules',
    'energy_ram_joules', 'energy_consumption', 'carbon_footprint',
    'gpu_used', 'gpu_memory', 'gpu_utilization', 'cuda_memory',
    'network_sent_mb', 'network_recv_mb', 'bytes_sent', 'bytes_received',
    'packets_sent', 'packets_received',
    'process_count', 'thread_count', 'context_switches', 'num_threads',
    'num_processes', 'child_processes',
    'cache_misses', 'cache_hits', 'instructions_retired', 'cycles_elapsed',
    'branch_misses', 'page_faults', 'tlb_misses',
    'runtime_errors', 'memory_errors', 'exception_count', 'error_count',
    'warning_count', 'num_exceptions',
    'output_size', 'return_value', 'stdout_length', 'stderr_length',
    'output_lines', 'print_statements_executed',
    'system_load', 'available_memory', 'free_disk', 'temperature'
]

METADATA_COLS = [
    'file_path', 'file_name', 'repo_name', 'repo_url', 'timestamp',
    'executed_successfully', 'syntax_valid', 'run_id', 'index', 'id',
    'file_id', 'repository', 'author', 'date_created', 'date_modified'
]

TARGET = 'energy_consumption_joules'

# ============================================================================
# STEP 1: LOAD AND FILTER
# ============================================================================
print("=" * 80)
print("ENHANCED STATIC FEATURES MODEL - MAXIMUM R² OPTIMIZATION")
print("=" * 80)

df = pd.read_csv(DATASET_PATH)
print(f"\n📊 Initial Dataset: {df.shape[0]:,} rows × {df.shape[1]} columns")

# Filter for successfully executed files
if 'executed_successfully' in df.columns:
    df = df[df['executed_successfully'] == True].copy()
    print(f"✅ After filtering executed_successfully: {df.shape[0]:,} rows")

# Remove zero/NaN energy
df = df[df[TARGET] > 0].copy()
df = df.dropna(subset=[TARGET])
print(f"✅ After removing zero/NaN energy: {df.shape[0]:,} rows")

# ============================================================================
# STEP 2: IDENTIFY STATIC FEATURES (KEEP MORE FEATURES)
# ============================================================================
print(f"\n🔍 Identifying static features with RELAXED filtering...")

all_cols = df.columns.tolist()

# First pass: Remove obvious dynamic and metadata
remaining_cols = [
    col for col in all_cols
    if col not in DYNAMIC_FEATURES + METADATA_COLS + [TARGET]
]

# Second pass: Only remove columns with STRONG dynamic keywords
# BE MORE PERMISSIVE - keep features like avg_indentation, max_nesting_depth
strong_dynamic_keywords = [
    'execution', 'runtime', 'elapsed',  # time-related
    'memory_', 'ram_', 'heap_',  # memory-related (but not 'memory' alone)
    'cpu_percent', 'cpu_usage', 'cpu_time',  # cpu metrics
    'disk_read', 'disk_write', 'io_read', 'io_write',  # disk I/O
    'network_', 'bytes_sent', 'bytes_received',  # network
    'process_count', 'thread_count',  # process metrics
    'exception_count', 'error_count',  # runtime errors
    'gpu_', 'cuda_',  # gpu
    'cache_miss', 'cache_hit', 'instructions_retired'  # hardware counters
]

static_features = []
removed_suspicious = []

for col in remaining_cols:
    col_lower = col.lower()
    is_dynamic = False

    for keyword in strong_dynamic_keywords:
        if keyword in col_lower:
            is_dynamic = True
            removed_suspicious.append(col)
            break

    if not is_dynamic:
        static_features.append(col)

print(f"\n📋 Feature Filtering (Relaxed):")
print(f"   Removed explicit dynamic: {len([c for c in all_cols if c in DYNAMIC_FEATURES])}")
print(f"   Removed suspicious: {len(removed_suspicious)}")
print(f"   ✅ STATIC FEATURES: {len(static_features)}")

X = df[static_features].copy()
y = df[TARGET].copy()

print(f"\n📐 Feature Matrix: {X.shape[0]:,} rows × {X.shape[1]} features")

# ============================================================================
# STEP 3: LESS AGGRESSIVE CLEANING
# ============================================================================
print(f"\n🧹 Data cleaning...")

# Handle infinite values
X.replace([np.inf, -np.inf], np.nan, inplace=True)

# Remove columns with >70% missing (more permissive)
missing_pct = X.isna().sum() / len(X)
cols_to_drop = missing_pct[missing_pct > 0.7].index.tolist()
if cols_to_drop:
    print(f"   Dropping {len(cols_to_drop)} columns with >70% missing")
    X = X.drop(columns=cols_to_drop)

# Fill NaN with median
for col in X.columns:
    if X[col].dtype in ['float64', 'int64', 'float32', 'int32']:
        if X[col].isna().sum() > 0:
            X[col].fillna(X[col].median(), inplace=True)

# Remove constant features only
constant_cols = [col for col in X.columns if X[col].nunique() <= 1]
if constant_cols:
    X = X.drop(columns=constant_cols)
    print(f"   Removed {len(constant_cols)} constant features")

# Keep more features - only remove bottom 2% variance
from sklearn.feature_selection import VarianceThreshold
if X.shape[1] > 10:
    variance_threshold = VarianceThreshold(threshold=0.0)
    variance_threshold.fit(X)
    feature_variances = variance_threshold.variances_
    variance_cutoff = np.percentile(feature_variances, 2)  # Changed from 5%
    low_var_mask = feature_variances > variance_cutoff
    X = X.loc[:, low_var_mask]
    print(f"   After variance filtering: {X.shape[1]} features")

print(f"✅ Features after cleaning: {X.shape[1]}")

# ============================================================================
# STEP 4: SMARTER OUTLIER REMOVAL (LESS AGGRESSIVE)
# ============================================================================
print(f"\n🔍 Smarter Outlier Removal...")
print(f"   Initial size: {X.shape[0]:,} samples")

# Only remove 5% most anomalous (was 10%)
iso_forest = IsolationForest(
    contamination=0.05,  # Less aggressive
    random_state=42,
    n_estimators=200,
    max_samples='auto'
)
outlier_pred = iso_forest.fit_predict(X)
mask_iso = outlier_pred == 1

X_clean = X[mask_iso].copy()
y_clean = y[mask_iso].copy()
print(f"   After IsolationForest (5%): {X_clean.shape[0]:,} samples")

# Less aggressive quantile filtering (2%-98%)
q_low = y_clean.quantile(0.02)  # Changed from 0.01
q_high = y_clean.quantile(0.98)  # Changed from 0.97
mask_quantile = (y_clean >= q_low) & (y_clean <= q_high)

X_clean = X_clean[mask_quantile]
y_clean = y_clean[mask_quantile]
print(f"   After quantile filtering (2%-98%): {X_clean.shape[0]:,} samples")

# More permissive z-score (4.0 instead of 3.5)
z_scores = np.abs((X_clean - X_clean.mean()) / (X_clean.std() + 1e-10))
mask_z = (z_scores < 4.0).all(axis=1)

X_clean = X_clean[mask_z]
y_clean = y_clean[mask_z]
print(f"   After z-score filtering (4.0σ): {X_clean.shape[0]:,} samples")
print(f"✅ Outliers removed: {X.shape[0] - X_clean.shape[0]:,} ({(1-X_clean.shape[0]/X.shape[0])*100:.1f}%)")

# ============================================================================
# STEP 5: FEATURE ENGINEERING
# ============================================================================
print(f"\n🔧 Feature Engineering...")

X_eng = X_clean.copy()

# Polynomial features for top predictors
if 'total_lines' in X_eng.columns and 'cyclomatic_complexity' in X_eng.columns:
    X_eng['lines_x_complexity'] = X_eng['total_lines'] * X_eng['cyclomatic_complexity']

if 'code_lines' in X_eng.columns and 'total_ast_nodes' in X_eng.columns:
    X_eng['code_x_ast'] = X_eng['code_lines'] * X_eng['total_ast_nodes']

if 'total_lines' in X_eng.columns:
    X_eng['lines_squared'] = X_eng['total_lines'] ** 2
    X_eng['lines_log'] = np.log1p(X_eng['total_lines'])

if 'cyclomatic_complexity' in X_eng.columns:
    X_eng['complexity_squared'] = X_eng['cyclomatic_complexity'] ** 2
    X_eng['complexity_log'] = np.log1p(X_eng['cyclomatic_complexity'])

# Ratios
if 'code_lines' in X_eng.columns and 'total_lines' in X_eng.columns:
    X_eng['code_to_total_ratio'] = X_eng['code_lines'] / (X_eng['total_lines'] + 1)

if 'comment_lines' in X_eng.columns and 'code_lines' in X_eng.columns:
    X_eng['comment_to_code_ratio'] = X_eng['comment_lines'] / (X_eng['code_lines'] + 1)

if 'cyclomatic_complexity' in X_eng.columns and 'total_lines' in X_eng.columns:
    X_eng['complexity_per_line_ratio'] = X_eng['cyclomatic_complexity'] / (X_eng['total_lines'] + 1)

print(f"✅ Engineered features: {X_eng.shape[1]} total features")

# ============================================================================
# STEP 6: LOG TRANSFORM TARGET
# ============================================================================
y_log = np.log1p(y_clean)
print(f"\n📈 Log-transformed target: [{y_log.min():.2f}, {y_log.max():.2f}]")

# ============================================================================
# STEP 7: FEATURE SELECTION - Keep Best Features
# ============================================================================
print(f"\n🎯 Feature Selection...")

# Use mutual information to select best features
selector = SelectKBest(score_func=mutual_info_regression, k=min(45, X_eng.shape[1]))
selector.fit(X_eng, y_log)

# Get selected features
selected_mask = selector.get_support()
X_selected = X_eng.loc[:, selected_mask]
selected_features = X_eng.columns[selected_mask].tolist()

print(f"✅ Selected top {X_selected.shape[1]} features using mutual information")

# ============================================================================
# STEP 8: POWER TRANSFORM + SCALING
# ============================================================================
# PowerTransformer makes features more Gaussian
power_transformer = PowerTransformer(method='yeo-johnson', standardize=True)
X_transformed = pd.DataFrame(
    power_transformer.fit_transform(X_selected),
    columns=X_selected.columns,
    index=X_selected.index
)

print(f"✅ Applied PowerTransformer (Yeo-Johnson) + standardization")

# ============================================================================
# STEP 9: TRAIN-TEST SPLIT
# ============================================================================
X_train, X_test, y_train, y_test = train_test_split(
    X_transformed, y_log, test_size=0.2, random_state=42, shuffle=True
)

print(f"\n📊 Train-Test Split:")
print(f"   Training: {X_train.shape[0]:,} samples")
print(f"   Test: {X_test.shape[0]:,} samples")

# ============================================================================
# STEP 10: TRAIN OPTIMIZED MODEL WITH REGULARIZATION
# ============================================================================
print(f"\n🚀 Training Regularized LightGBM...")

# More regularization to prevent overfitting
model = LGBMRegressor(
    objective='regression',
    n_estimators=1500,  # Reduced
    learning_rate=0.005,  # Lower learning rate
    num_leaves=20,  # Reduced from 31
    max_depth=6,  # Reduced from 8
    min_child_samples=30,  # Increased
    min_child_weight=0.01,
    subsample=0.7,  # More aggressive subsampling
    colsample_bytree=0.7,
    reg_alpha=1.0,  # Stronger L1 regularization
    reg_lambda=1.0,  # Stronger L2 regularization
    min_split_gain=0.01,
    random_state=42,
    verbose=-1,
    n_jobs=-1,
    force_col_wise=True
)

model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_metric='rmse'
)

# ============================================================================
# STEP 11: EVALUATE
# ============================================================================
print(f"\n" + "=" * 80)
print("MODEL PERFORMANCE")
print("=" * 80)

y_train_pred_log = model.predict(X_train)
y_test_pred_log = model.predict(X_test)

r2_train_log = r2_score(y_train, y_train_pred_log)
r2_test_log = r2_score(y_test, y_test_pred_log)

print(f"\n📊 Log-Scale Performance:")
print(f"   Training R²: {r2_train_log:.6f}")
print(f"   Test R²:     {r2_test_log:.6f}")
print(f"   Overfitting Gap: {r2_train_log - r2_test_log:.6f}")

# Original scale
y_train_actual = np.expm1(y_train)
y_train_pred = np.expm1(y_train_pred_log)
y_test_actual = np.expm1(y_test)
y_test_pred = np.expm1(y_test_pred_log)

r2_train_original = r2_score(y_train_actual, y_train_pred)
r2_test_original = r2_score(y_test_actual, y_test_pred)

print(f"\n⚡ Original Joules Scale:")
print(f"   Training R²: {r2_train_original:.6f}")
print(f"   Test R²:     {r2_test_original:.6f}")

mae_test = mean_absolute_error(y_test_actual, y_test_pred)
rmse_test = np.sqrt(mean_squared_error(y_test_actual, y_test_pred))
mean_energy = y_test_actual.mean()

print(f"\n📏 Error Metrics:")
print(f"   Test MAE:  {mae_test:,.2f} J ({(mae_test/mean_energy)*100:.1f}% of mean)")
print(f"   Test RMSE: {rmse_test:,.2f} J")

# ============================================================================
# STEP 12: CROSS-VALIDATION
# ============================================================================
print(f"\n🔄 5-Fold Cross-Validation:")
cv_scores = cross_val_score(
    model, X_transformed, y_log, cv=5, scoring='r2', n_jobs=-1
)
print(f"   CV R² Scores: {[f'{s:.4f}' for s in cv_scores]}")
print(f"   Mean CV R²: {cv_scores.mean():.6f} (±{cv_scores.std():.6f})")

# ============================================================================
# STEP 13: FEATURE IMPORTANCE
# ============================================================================
print(f"\n🎯 Top 15 Most Important Features:")
print("-" * 60)

feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

for idx, row in feature_importance.head(15).iterrows():
    print(f"   {row['feature']:35s} {row['importance']:8.1f}")

# ============================================================================
# FINAL SUMMARY
# ============================================================================
print(f"\n" + "=" * 80)
print("🎉 FINAL RESULTS")
print("=" * 80)
print(f"✅ Training samples: {X_train.shape[0]:,}")
print(f"✅ Features used: {X_transformed.shape[1]}")
print(f"✅ Test R² (Log): {r2_test_log:.6f}")
print(f"✅ Test R² (Original): {r2_test_original:.6f}")
print(f"✅ CV R²: {cv_scores.mean():.6f} ±{cv_scores.std():.4f}")
print(f"✅ Test MAE: {mae_test:,.2f} J ({(mae_test/mean_energy)*100:.1f}% of mean)")
print(f"✅ Overfitting: {r2_train_log - r2_test_log:.4f} (lower is better)")
print("=" * 80)

print(f"\n💡 Key Improvements:")
print(f"   • Less aggressive outlier removal (kept more data)")
print(f"   • Relaxed feature filtering (kept useful features)")
print(f"   • Feature engineering (interactions, polynomials)")
print(f"   • Feature selection (mutual information)")
print(f"   • PowerTransformer for better distributions")
print(f"   • Stronger regularization (reduced overfitting)")

ENHANCED STATIC FEATURES MODEL - MAXIMUM R² OPTIMIZATION

📊 Initial Dataset: 6,711 rows × 107 columns
✅ After filtering executed_successfully: 1,567 rows
✅ After removing zero/NaN energy: 1,567 rows

🔍 Identifying static features with RELAXED filtering...

📋 Feature Filtering (Relaxed):
   Removed explicit dynamic: 4
   Removed suspicious: 3
   ✅ STATIC FEATURES: 95

📐 Feature Matrix: 1,567 rows × 95 features

🧹 Data cleaning...
   Dropping 2 columns with >70% missing
   Removed 3 constant features
   After variance filtering: 88 features
✅ Features after cleaning: 88

🔍 Smarter Outlier Removal...
   Initial size: 1,567 samples
   After IsolationForest (5%): 1,488 samples
   After quantile filtering (2%-98%): 1,428 samples
   After z-score filtering (4.0σ): 1,089 samples
✅ Outliers removed: 478 (30.5%)

🔧 Feature Engineering...
✅ Engineered features: 97 total features

📈 Log-transformed target: [1.47, 6.11]

🎯 Feature Selection...
✅ Selected top 45 features using mutual information
✅ A

In [ ]:
# @title
import pandas as pd
import numpy as np
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge, ElasticNet
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION
# ============================================================================
DATASET_PATH = '/content/ml_code_dataset.csv'

# ============================================================================
# FEATURE CATEGORIZATION - STRICT LEAKAGE PREVENTION
# ============================================================================

# TARGET (what we're predicting)
TARGET = 'energy_consumption_joules'

# LEAKAGE FEATURES - MUST BE REMOVED
# These are derived FROM the target or highly correlated outcomes
LEAKAGE_FEATURES = [
    'co2_emissions_g',           # Direct calculation from energy
    'energy_per_line',           # Derived: energy / lines
    'time_per_line',             # Derived: time / lines
    'co2_per_line',              # Derived: co2 / lines
    'energy_pkg_joules',         # Alternative energy measurement
    'energy_ram_joules',         # Alternative energy measurement
    'power_draw_watts',          # Direct energy measurement
]

# DYNAMIC FEATURES - CAN USE (measured during execution, but not derived from energy)
DYNAMIC_FEATURES = [
    'execution_time',            # Independent measurement
    'cpu_percent_avg',           # Independent measurement
    'cpu_percent_max',           # Independent measurement
    'memory_usage_mb',           # Independent measurement
    'memory_peak_mb',            # Independent measurement
]

# GRAY FEATURES - CAN USE
GRAY_FEATURES = [
    'executed_successfully', 'execution_error', 'exit_code',
    'syntax_valid', 'parse_error', 'syntax_error_line', 'syntax_error_type'
]

# STATIC FEATURES - CAN USE
STATIC_FEATURES = [
    'total_lines', 'code_lines', 'blank_lines', 'total_characters',
    'file_size_bytes', 'file_size_kb', 'avg_line_length', 'max_line_length',
    'min_line_length', 'std_line_length', 'comment_lines', 'single_quotes',
    'double_quotes', 'triple_quotes', 'total_quotes', 'parentheses_count',
    'brackets_count', 'braces_count', 'semicolon_count', 'colon_count',
    'unique_indentation_levels', 'avg_indentation', 'max_indentation',
    'std_indentation', 'keyword_def_count', 'keyword_class_count',
    'keyword_import_count', 'keyword_from_count', 'keyword_if_count',
    'keyword_else_count', 'keyword_elif_count', 'keyword_for_count',
    'keyword_while_count', 'keyword_try_count', 'keyword_except_count',
    'keyword_finally_count', 'keyword_with_count', 'keyword_return_count',
    'keyword_yield_count', 'keyword_lambda_count', 'keyword_async_count',
    'keyword_await_count', 'keyword_global_count', 'keyword_nonlocal_count',
    'functions_count', 'async_functions_count', 'classes_count',
    'imports_count', 'if_statements', 'for_loops', 'while_loops',
    'async_for_loops', 'try_blocks', 'with_statements', 'async_with_statements',
    'lambda_functions', 'list_comprehensions', 'dict_comprehensions',
    'set_comprehensions', 'generator_expressions', 'binary_operations',
    'unary_operations', 'boolean_operations', 'comparisons', 'decorators',
    'assignments', 'augmented_assignments', 'return_statements',
    'yield_statements', 'raise_statements', 'assert_statements',
    'cyclomatic_complexity', 'max_nesting_depth', 'unique_function_names',
    'unique_class_names', 'unique_imports', 'unique_variables',
    'total_ast_nodes', 'comment_ratio', 'control_flow_ratio',
    'function_ratio', 'class_ratio', 'code_density', 'complexity_per_line',
    'functions_per_line', 'avg_complexity_per_function'
]

# METADATA - EXCLUDE
METADATA_COLS = [
    'file_path', 'file_name', 'repo_name', 'repo_url', 'timestamp',
    'run_id', 'index', 'id', 'file_id', 'repository', 'author',
    'date_created', 'date_modified', 'file_index'
]

# ============================================================================
# STEP 1: LOAD DATA
# ============================================================================
print("=" * 80)
print("STATIC + DYNAMIC FEATURES MODEL (ZERO LEAKAGE)")
print("=" * 80)

df = pd.read_csv(DATASET_PATH)
print(f"\n📊 Initial Dataset: {df.shape[0]:,} rows × {df.shape[1]} columns")

if 'executed_successfully' in df.columns:
    df = df[df['executed_successfully'] == True].copy()

df = df[df[TARGET] > 0].copy()
df = df.dropna(subset=[TARGET])
print(f"✅ Valid samples: {df.shape[0]:,} rows")

# ============================================================================
# STEP 2: BUILD FEATURE SET (PREVENT LEAKAGE!)
# ============================================================================
print(f"\n🔒 Building feature set with STRICT leakage prevention...")

available_cols = df.columns.tolist()

# Collect all usable features
usable_features = []
for col in available_cols:
    # Include if it's static, dynamic, or gray
    if col in STATIC_FEATURES or col in DYNAMIC_FEATURES or col in GRAY_FEATURES:
        # Exclude if it's target, metadata, or leakage
        if col not in [TARGET] + METADATA_COLS + LEAKAGE_FEATURES:
            usable_features.append(col)

# Categorize features
static_used = [f for f in usable_features if f in STATIC_FEATURES]
dynamic_used = [f for f in usable_features if f in DYNAMIC_FEATURES]
gray_used = [f for f in usable_features if f in GRAY_FEATURES]

print(f"\n📋 Feature Breakdown:")
print(f"   Static features: {len(static_used)}")
print(f"   Dynamic features: {len(dynamic_used)}")
print(f"   Gray features: {len(gray_used)}")
print(f"   ✅ Total usable: {len(usable_features)}")

# Verify no leakage
leaked = [f for f in usable_features if f in LEAKAGE_FEATURES]
if leaked:
    print(f"   ❌ ERROR: Leaked features found: {leaked}")
    exit()
else:
    print(f"   ✅ No leakage detected!")

# Show dynamic features being used
if dynamic_used:
    print(f"\n🔥 Dynamic features included:")
    for feat in dynamic_used:
        print(f"      • {feat}")

X = df[usable_features].copy()
y = df[TARGET].copy()

print(f"\n📐 Feature Matrix: {X.shape[0]:,} rows × {X.shape[1]} features")

# ============================================================================
# STEP 3: MINIMAL CLEANING
# ============================================================================
print(f"\n🧹 Data cleaning...")

# Handle infinite/NaN
X.replace([np.inf, -np.inf], np.nan, inplace=True)
for col in X.columns:
    if X[col].dtype in ['float64', 'int64']:
        if X[col].isna().sum() > 0:
            X[col].fillna(X[col].median(), inplace=True)

# Remove constant features
constant_cols = [col for col in X.columns if X[col].nunique() <= 1]
if constant_cols:
    X = X.drop(columns=constant_cols)
    print(f"   Removed {len(constant_cols)} constant features")

# Very minimal outlier removal
q_low = y.quantile(0.005)
q_high = y.quantile(0.995)
mask = (y >= q_low) & (y <= q_high)
X = X[mask]
y = y[mask]

print(f"✅ Clean data: {X.shape[0]:,} samples, {X.shape[1]} features")

# ============================================================================
# STEP 4: FEATURE ENGINEERING
# ============================================================================
print(f"\n🔧 Feature Engineering...")

X_eng = X.copy()

# === INTERACTIONS BETWEEN STATIC AND DYNAMIC ===
print("   • Static × Dynamic interactions...")

# Execution time interactions
if 'execution_time' in X_eng.columns:
    if 'total_lines' in X_eng.columns:
        X_eng['time_per_line_calc'] = X_eng['execution_time'] / (X_eng['total_lines'] + 1)
    if 'cyclomatic_complexity' in X_eng.columns:
        X_eng['time_per_complexity'] = X_eng['execution_time'] / (X_eng['cyclomatic_complexity'] + 1)
    if 'for_loops' in X_eng.columns:
        X_eng['time_per_loop'] = X_eng['execution_time'] / (X_eng['for_loops'] + 1)

    # Time efficiency ratios
    X_eng['time_squared'] = X_eng['execution_time'] ** 2
    X_eng['time_log'] = np.log1p(X_eng['execution_time'])
    X_eng['time_sqrt'] = np.sqrt(X_eng['execution_time'])

# Memory interactions
if 'memory_peak_mb' in X_eng.columns:
    if 'total_lines' in X_eng.columns:
        X_eng['memory_per_line'] = X_eng['memory_peak_mb'] / (X_eng['total_lines'] + 1)
    if 'unique_variables' in X_eng.columns:
        X_eng['memory_per_var'] = X_eng['memory_peak_mb'] / (X_eng['unique_variables'] + 1)
    if 'classes_count' in X_eng.columns:
        X_eng['memory_per_class'] = X_eng['memory_peak_mb'] / (X_eng['classes_count'] + 1)

    X_eng['memory_squared'] = X_eng['memory_peak_mb'] ** 2
    X_eng['memory_log'] = np.log1p(X_eng['memory_peak_mb'])

# CPU interactions
if 'cpu_percent_avg' in X_eng.columns:
    if 'execution_time' in X_eng.columns:
        X_eng['cpu_time_product'] = X_eng['cpu_percent_avg'] * X_eng['execution_time']
    if 'cyclomatic_complexity' in X_eng.columns:
        X_eng['cpu_complexity_ratio'] = X_eng['cpu_percent_avg'] / (X_eng['cyclomatic_complexity'] + 1)

if 'cpu_percent_max' in X_eng.columns and 'cpu_percent_avg' in X_eng.columns:
    X_eng['cpu_variability'] = X_eng['cpu_percent_max'] - X_eng['cpu_percent_avg']

# === COMBINED ENERGY PROXY ===
print("   • Combined energy proxy...")

energy_proxy = 0
if 'execution_time' in X_eng.columns:
    energy_proxy += X_eng['execution_time'] * 50  # Time is major factor

if 'memory_peak_mb' in X_eng.columns:
    energy_proxy += X_eng['memory_peak_mb'] * 0.5  # Memory contributes

if 'cpu_percent_avg' in X_eng.columns:
    energy_proxy += X_eng['cpu_percent_avg'] * 2  # CPU contributes

if isinstance(energy_proxy, pd.Series):
    X_eng['combined_energy_proxy'] = energy_proxy

# === STATIC FEATURE ENGINEERING ===
print("   • Static feature polynomials...")

for col in ['total_lines', 'code_lines', 'cyclomatic_complexity', 'total_ast_nodes']:
    if col in X_eng.columns:
        X_eng[f'{col}_sqrt'] = np.sqrt(X_eng[col] + 1)
        X_eng[f'{col}_log'] = np.log1p(X_eng[col])
        X_eng[f'{col}_square'] = X_eng[col] ** 2

# Static interactions
if 'total_lines' in X_eng.columns and 'cyclomatic_complexity' in X_eng.columns:
    X_eng['lines_x_complexity'] = X_eng['total_lines'] * X_eng['cyclomatic_complexity']

if 'code_lines' in X_eng.columns and 'total_ast_nodes' in X_eng.columns:
    X_eng['code_x_ast'] = X_eng['code_lines'] * X_eng['total_ast_nodes']

# Ratios
if 'code_lines' in X_eng.columns and 'total_lines' in X_eng.columns:
    X_eng['code_ratio'] = X_eng['code_lines'] / (X_eng['total_lines'] + 1)

if 'comment_lines' in X_eng.columns and 'code_lines' in X_eng.columns:
    X_eng['comment_ratio_calc'] = X_eng['comment_lines'] / (X_eng['code_lines'] + 1)

print(f"✅ Engineered features: {X_eng.shape[1]} total")

# ============================================================================
# STEP 5: LOG TRANSFORM TARGET
# ============================================================================
y_log = np.log1p(y)
print(f"\n📈 Target (log): [{y_log.min():.2f}, {y_log.max():.2f}]")

# ============================================================================
# STEP 6: FEATURE SELECTION
# ============================================================================
print(f"\n🎯 Feature Selection...")

k_best = min(80, X_eng.shape[1])
selector = SelectKBest(score_func=mutual_info_regression, k=k_best)
selector.fit(X_eng, y_log)
X_selected = X_eng.loc[:, selector.get_support()]

print(f"✅ Selected {X_selected.shape[1]} features")

# ============================================================================
# STEP 7: NORMALIZE
# ============================================================================
qt = QuantileTransformer(
    n_quantiles=min(1000, X_selected.shape[0]),
    output_distribution='normal',
    random_state=42
)
X_transformed = pd.DataFrame(
    qt.fit_transform(X_selected),
    columns=X_selected.columns,
    index=X_selected.index
)

# ============================================================================
# STEP 8: STRATIFIED SPLIT
# ============================================================================
y_bins = pd.qcut(y_log, q=5, labels=False, duplicates='drop')
X_train, X_test, y_train, y_test = train_test_split(
    X_transformed, y_log, test_size=0.15, random_state=42,
    stratify=y_bins, shuffle=True
)

print(f"\n📊 Split: {X_train.shape[0]:,} train, {X_test.shape[0]:,} test")

# ============================================================================
# STEP 9: OPTIMIZED STACKING ENSEMBLE
# ============================================================================
print(f"\n🚀 Training Stacking Ensemble (Static + Dynamic Features)...")

# With dynamic features, we can use less regularization
base_models = [
    ('lgbm1', LGBMRegressor(
        n_estimators=1500, learning_rate=0.01, num_leaves=50,
        max_depth=10, min_child_samples=10, subsample=0.9,
        colsample_bytree=0.9, reg_alpha=0.1, reg_lambda=0.1,
        random_state=42, verbose=-1, n_jobs=-1
    )),
    ('lgbm2', LGBMRegressor(
        n_estimators=1200, learning_rate=0.015, num_leaves=35,
        max_depth=8, min_child_samples=15, subsample=0.85,
        colsample_bytree=0.85, reg_alpha=0.3, reg_lambda=0.3,
        random_state=43, verbose=-1, n_jobs=-1
    )),
    ('xgb1', XGBRegressor(
        n_estimators=1500, learning_rate=0.01, max_depth=10,
        min_child_weight=1, subsample=0.9, colsample_bytree=0.9,
        reg_alpha=0.1, reg_lambda=0.1, random_state=42,
        verbosity=0, n_jobs=-1
    )),
    ('xgb2', XGBRegressor(
        n_estimators=1200, learning_rate=0.015, max_depth=8,
        min_child_weight=2, subsample=0.85, colsample_bytree=0.85,
        reg_alpha=0.3, reg_lambda=0.3, random_state=43,
        verbosity=0, n_jobs=-1
    )),
    ('ridge', Ridge(alpha=1.0)),
    ('elastic', ElasticNet(alpha=0.001, l1_ratio=0.5, max_iter=5000))
]

meta_model = Ridge(alpha=1.0)

stacking_model = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5,
    n_jobs=-1
)

print("   Training ensemble...")
stacking_model.fit(X_train, y_train)

# ============================================================================
# STEP 10: EVALUATE
# ============================================================================
print(f"\n" + "=" * 80)
print("MODEL PERFORMANCE: STATIC + DYNAMIC (NO LEAKAGE)")
print("=" * 80)

y_train_pred_log = stacking_model.predict(X_train)
y_test_pred_log = stacking_model.predict(X_test)

r2_train_log = r2_score(y_train, y_train_pred_log)
r2_test_log = r2_score(y_test, y_test_pred_log)

print(f"\n📊 Log-Scale Performance:")
print(f"   Training R²: {r2_train_log:.6f}")
print(f"   Test R²:     {r2_test_log:.6f}")
print(f"   Overfitting Gap: {r2_train_log - r2_test_log:.6f}")

# Original scale
y_train_actual = np.expm1(y_train)
y_train_pred = np.expm1(y_train_pred_log)
y_test_actual = np.expm1(y_test)
y_test_pred = np.expm1(y_test_pred_log)

r2_train_original = r2_score(y_train_actual, y_train_pred)
r2_test_original = r2_score(y_test_actual, y_test_pred)

print(f"\n⚡ Original Joules Scale:")
print(f"   Training R²: {r2_train_original:.6f}")
print(f"   Test R²:     {r2_test_original:.6f}")

mae_test = mean_absolute_error(y_test_actual, y_test_pred)
rmse_test = np.sqrt(mean_squared_error(y_test_actual, y_test_pred))
mean_energy = y_test_actual.mean()

print(f"\n📏 Error Metrics:")
print(f"   Test MAE:  {mae_test:,.2f} J ({(mae_test/mean_energy)*100:.1f}% of mean)")
print(f"   Test RMSE: {rmse_test:,.2f} J")
print(f"   Mean Energy: {mean_energy:,.2f} J")

# ============================================================================
# STEP 11: CROSS-VALIDATION
# ============================================================================
print(f"\n🔄 5-Fold Cross-Validation:")
cv_scores = cross_val_score(
    stacking_model, X_transformed, y_log, cv=5,
    scoring='r2', n_jobs=-1
)
print(f"   CV R²: {[f'{s:.4f}' for s in cv_scores]}")
print(f"   Mean: {cv_scores.mean():.6f} (±{cv_scores.std():.6f})")

# ============================================================================
# STEP 12: FEATURE IMPORTANCE ANALYSIS
# ============================================================================
print(f"\n🎯 Top 20 Features (Static vs Dynamic):")
print("-" * 80)

# Get importance from LGBM
lgbm_model = base_models[0][1]
lgbm_model.fit(X_train, y_train)

feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': lgbm_model.feature_importances_
}).sort_values('importance', ascending=False)

for idx, row in feature_importance.head(20).iterrows():
    feat = row['feature']
    # Mark feature type
    if any(d in feat for d in ['execution_time', 'memory', 'cpu']):
        marker = "🔥 [DYNAMIC]"
    elif any(s in feat for s in ['time_per', 'memory_per', 'cpu_']):
        marker = "⚡ [HYBRID]"
    else:
        marker = "📊 [STATIC] "

    print(f"   {marker} {feat:40s} {row['importance']:8.1f}")

# ============================================================================
# STEP 13: DYNAMIC FEATURE CONTRIBUTION ANALYSIS
# ============================================================================
print(f"\n🔬 Dynamic Feature Impact Analysis:")

# Count dynamic features in top 20
top_20_features = feature_importance.head(20)['feature'].tolist()
dynamic_in_top = sum(1 for f in top_20_features
                     if any(d in f for d in ['execution_time', 'memory', 'cpu']))
hybrid_in_top = sum(1 for f in top_20_features
                    if any(h in f for h in ['time_per', 'memory_per', 'cpu_']))

print(f"   Dynamic features in Top 20: {dynamic_in_top}")
print(f"   Hybrid features in Top 20: {hybrid_in_top}")
print(f"   Static features in Top 20: {20 - dynamic_in_top - hybrid_in_top}")

# Total importance by type
total_importance = feature_importance['importance'].sum()
dynamic_importance = feature_importance[
    feature_importance['feature'].str.contains('execution_time|memory_|cpu_', regex=True)
]['importance'].sum()

print(f"\n   Dynamic feature importance: {dynamic_importance:,.0f} ({dynamic_importance/total_importance*100:.1f}%)")
print(f"   Static feature importance: {total_importance - dynamic_importance:,.0f} ({(1-dynamic_importance/total_importance)*100:.1f}%)")

# ============================================================================
# STEP 14: COMPARE WITH STATIC-ONLY
# ============================================================================
print(f"\n📊 Performance Comparison:")
print(f"   Static-only model (previous): R² ~ 0.53-0.65")
print(f"   Static + Dynamic (current):   R² = {r2_test_log:.4f}")
print(f"   Improvement: +{(r2_test_log - 0.59) * 100:.1f} percentage points")

# ============================================================================
# FINAL SUMMARY
# ============================================================================
print(f"\n" + "=" * 80)
print("🎉 FINAL RESULTS - STATIC + DYNAMIC FEATURES")
print("=" * 80)
print(f"✅ Samples: {X_train.shape[0]:,} train, {X_test.shape[0]:,} test")
print(f"✅ Features: {X_transformed.shape[1]} (static + dynamic, zero leakage)")
print(f"✅ Dynamic features used: {len(dynamic_used)}")
print(f"✅ Test R² (Log): {r2_test_log:.6f}")
print(f"✅ Test R² (Original): {r2_test_original:.6f}")
print(f"✅ CV R²: {cv_scores.mean():.6f} ±{cv_scores.std():.4f}")
print(f"✅ MAE: {mae_test:,.2f} J ({(mae_test/mean_energy)*100:.1f}%)")
print(f"✅ Overfitting: {r2_train_log - r2_test_log:.4f}")
print("=" * 80)

print(f"\n🔒 Leakage Prevention Summary:")
print(f"   ✅ Excluded: {len(LEAKAGE_FEATURES)} leakage features")
print(f"   ✅ Included: {len(dynamic_used)} clean dynamic features")
print(f"   ✅ No energy-derived features used")

if r2_test_log >= 0.9:
    print("\n🎊 EXCELLENT: R² ≥ 0.90 - Near-perfect prediction!")
elif r2_test_log >= 0.8:
    print("\n🚀 OUTSTANDING: R² ≥ 0.80 - Production-ready!")
elif r2_test_log >= 0.7:
    print("\n✅ STRONG: R² ≥ 0.70 - Very good performance!")
else:
    print(f"\n📊 Current: R² = {r2_test_log:.3f}")

print(f"\n💡 Key Insights:")
print(f"   • Dynamic features explain {dynamic_importance/total_importance*100:.0f}% of prediction power")
print(f"   • execution_time likely most important predictor")
print(f"   • Model shows upper bound of predictability")
print(f"   • Can be used for post-execution profiling/optimization")

STATIC + DYNAMIC FEATURES MODEL (ZERO LEAKAGE)

📊 Initial Dataset: 6,711 rows × 107 columns
✅ Valid samples: 1,567 rows

🔒 Building feature set with STRICT leakage prevention...

📋 Feature Breakdown:
   Static features: 86
   Dynamic features: 5
   Gray features: 7
   ✅ Total usable: 98
   ✅ No leakage detected!

🔥 Dynamic features included:
      • execution_time
      • cpu_percent_avg
      • cpu_percent_max
      • memory_usage_mb
      • memory_peak_mb

📐 Feature Matrix: 1,567 rows × 98 features

🧹 Data cleaning...
   Removed 6 constant features
✅ Clean data: 1,551 samples, 92 features

🔧 Feature Engineering...
   • Static × Dynamic interactions...
   • Combined energy proxy...
   • Static feature polynomials...
✅ Engineered features: 123 total

📈 Target (log): [1.46, 6.18]

🎯 Feature Selection...
✅ Selected 80 features

📊 Split: 1,318 train, 233 test

🚀 Training Stacking Ensemble (Static + Dynamic Features)...
   Training ensemble...

MODEL PERFORMANCE: STATIC + DYNAMIC (NO LEAKA

In [ ]:
# @title
import pandas as pd
import numpy as np
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge, ElasticNet
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION
# ============================================================================
DATASET_PATH = '/content/Py_Raw.csv'

# ============================================================================
# FEATURE CATEGORIZATION - STRICT LEAKAGE PREVENTION
# ============================================================================

# TARGET (what we're predicting)
TARGET = 'energy_consumption_joules'

# LEAKAGE FEATURES - MUST BE REMOVED
# These are derived FROM the target or highly correlated outcomes
LEAKAGE_FEATURES = [
    'co2_emissions_g',           # Direct calculation from energy
    'energy_per_line',           # Derived: energy / lines
    'time_per_line',             # Derived: time / lines
    'co2_per_line',              # Derived: co2 / lines
    'energy_pkg_joules',         # Alternative energy measurement
    'energy_ram_joules',         # Alternative energy measurement
    'power_draw_watts',          # Direct energy measurement
]

# DYNAMIC FEATURES - CAN USE (measured during execution, but not derived from energy)
DYNAMIC_FEATURES = [
    'execution_time',            # Independent measurement
    'cpu_percent_avg',           # Independent measurement
    'cpu_percent_max',           # Independent measurement
    'memory_usage_mb',           # Independent measurement
    'memory_peak_mb',            # Independent measurement
]

# GRAY FEATURES - CAN USE
GRAY_FEATURES = [
    'executed_successfully', 'execution_error', 'exit_code',
    'syntax_valid', 'parse_error', 'syntax_error_line', 'syntax_error_type'
]

# STATIC FEATURES - CAN USE
STATIC_FEATURES = [
    'total_lines', 'code_lines', 'blank_lines', 'total_characters',
    'file_size_bytes', 'file_size_kb', 'avg_line_length', 'max_line_length',
    'min_line_length', 'std_line_length', 'comment_lines', 'single_quotes',
    'double_quotes', 'triple_quotes', 'total_quotes', 'parentheses_count',
    'brackets_count', 'braces_count', 'semicolon_count', 'colon_count',
    'unique_indentation_levels', 'avg_indentation', 'max_indentation',
    'std_indentation', 'keyword_def_count', 'keyword_class_count',
    'keyword_import_count', 'keyword_from_count', 'keyword_if_count',
    'keyword_else_count', 'keyword_elif_count', 'keyword_for_count',
    'keyword_while_count', 'keyword_try_count', 'keyword_except_count',
    'keyword_finally_count', 'keyword_with_count', 'keyword_return_count',
    'keyword_yield_count', 'keyword_lambda_count', 'keyword_async_count',
    'keyword_await_count', 'keyword_global_count', 'keyword_nonlocal_count',
    'functions_count', 'async_functions_count', 'classes_count',
    'imports_count', 'if_statements', 'for_loops', 'while_loops',
    'async_for_loops', 'try_blocks', 'with_statements', 'async_with_statements',
    'lambda_functions', 'list_comprehensions', 'dict_comprehensions',
    'set_comprehensions', 'generator_expressions', 'binary_operations',
    'unary_operations', 'boolean_operations', 'comparisons', 'decorators',
    'assignments', 'augmented_assignments', 'return_statements',
    'yield_statements', 'raise_statements', 'assert_statements',
    'cyclomatic_complexity', 'max_nesting_depth', 'unique_function_names',
    'unique_class_names', 'unique_imports', 'unique_variables',
    'total_ast_nodes', 'comment_ratio', 'control_flow_ratio',
    'function_ratio', 'class_ratio', 'code_density', 'complexity_per_line',
    'functions_per_line', 'avg_complexity_per_function'
]

# METADATA - EXCLUDE
METADATA_COLS = [
    'file_path', 'file_name', 'repo_name', 'repo_url', 'timestamp',
    'run_id', 'index', 'id', 'file_id', 'repository', 'author',
    'date_created', 'date_modified', 'file_index'
]

# ============================================================================
# STEP 1: LOAD DATA
# ============================================================================
print("=" * 80)
print("STATIC + DYNAMIC FEATURES MODEL (ZERO LEAKAGE)")
print("=" * 80)

df = pd.read_csv(DATASET_PATH)
print(f"\n📊 Initial Dataset: {df.shape[0]:,} rows × {df.shape[1]} columns")

if 'executed_successfully' in df.columns:
    df = df[df['executed_successfully'] == True].copy()

df = df[df[TARGET] > 0].copy()
df = df.dropna(subset=[TARGET])
print(f"✅ Valid samples: {df.shape[0]:,} rows")

# ============================================================================
# STEP 2: BUILD FEATURE SET (PREVENT LEAKAGE!)
# ============================================================================
print(f"\n🔒 Building feature set with STRICT leakage prevention...")

available_cols = df.columns.tolist()

# Collect all usable features
usable_features = []
for col in available_cols:
    # Include if it's static, dynamic, or gray
    if col in STATIC_FEATURES or col in DYNAMIC_FEATURES or col in GRAY_FEATURES:
        # Exclude if it's target, metadata, or leakage
        if col not in [TARGET] + METADATA_COLS + LEAKAGE_FEATURES:
            usable_features.append(col)

# Categorize features
static_used = [f for f in usable_features if f in STATIC_FEATURES]
dynamic_used = [f for f in usable_features if f in DYNAMIC_FEATURES]
gray_used = [f for f in usable_features if f in GRAY_FEATURES]

print(f"\n📋 Feature Breakdown:")
print(f"   Static features: {len(static_used)}")
print(f"   Dynamic features: {len(dynamic_used)}")
print(f"   Gray features: {len(gray_used)}")
print(f"   ✅ Total usable: {len(usable_features)}")

# Verify no leakage
leaked = [f for f in usable_features if f in LEAKAGE_FEATURES]
if leaked:
    print(f"   ❌ ERROR: Leaked features found: {leaked}")
    exit()
else:
    print(f"   ✅ No leakage detected!")

# Show dynamic features being used
if dynamic_used:
    print(f"\n🔥 Dynamic features included:")
    for feat in dynamic_used:
        print(f"      • {feat}")

X = df[usable_features].copy()
y = df[TARGET].copy()

print(f"\n📐 Feature Matrix: {X.shape[0]:,} rows × {X.shape[1]} features")

# ============================================================================
# STEP 3: MINIMAL CLEANING
# ============================================================================
print(f"\n🧹 Data cleaning...")

# Handle infinite/NaN
X.replace([np.inf, -np.inf], np.nan, inplace=True)
for col in X.columns:
    if X[col].dtype in ['float64', 'int64']:
        if X[col].isna().sum() > 0:
            X[col].fillna(X[col].median(), inplace=True)

# Remove constant features
constant_cols = [col for col in X.columns if X[col].nunique() <= 1]
if constant_cols:
    X = X.drop(columns=constant_cols)
    print(f"   Removed {len(constant_cols)} constant features")

# Very minimal outlier removal
q_low = y.quantile(0.005)
q_high = y.quantile(0.995)
mask = (y >= q_low) & (y <= q_high)
X = X[mask]
y = y[mask]

print(f"✅ Clean data: {X.shape[0]:,} samples, {X.shape[1]} features")

# ============================================================================
# STEP 4: FEATURE ENGINEERING
# ============================================================================
print(f"\n🔧 Feature Engineering...")

X_eng = X.copy()

# === INTERACTIONS BETWEEN STATIC AND DYNAMIC ===
print("   • Static × Dynamic interactions...")

# Execution time interactions
if 'execution_time' in X_eng.columns:
    if 'total_lines' in X_eng.columns:
        X_eng['time_per_line_calc'] = X_eng['execution_time'] / (X_eng['total_lines'] + 1)
    if 'cyclomatic_complexity' in X_eng.columns:
        X_eng['time_per_complexity'] = X_eng['execution_time'] / (X_eng['cyclomatic_complexity'] + 1)
    if 'for_loops' in X_eng.columns:
        X_eng['time_per_loop'] = X_eng['execution_time'] / (X_eng['for_loops'] + 1)

    # Time efficiency ratios
    X_eng['time_squared'] = X_eng['execution_time'] ** 2
    X_eng['time_log'] = np.log1p(X_eng['execution_time'])
    X_eng['time_sqrt'] = np.sqrt(X_eng['execution_time'])

# Memory interactions
if 'memory_peak_mb' in X_eng.columns:
    if 'total_lines' in X_eng.columns:
        X_eng['memory_per_line'] = X_eng['memory_peak_mb'] / (X_eng['total_lines'] + 1)
    if 'unique_variables' in X_eng.columns:
        X_eng['memory_per_var'] = X_eng['memory_peak_mb'] / (X_eng['unique_variables'] + 1)
    if 'classes_count' in X_eng.columns:
        X_eng['memory_per_class'] = X_eng['memory_peak_mb'] / (X_eng['classes_count'] + 1)

    X_eng['memory_squared'] = X_eng['memory_peak_mb'] ** 2
    X_eng['memory_log'] = np.log1p(X_eng['memory_peak_mb'])

# CPU interactions
if 'cpu_percent_avg' in X_eng.columns:
    if 'execution_time' in X_eng.columns:
        X_eng['cpu_time_product'] = X_eng['cpu_percent_avg'] * X_eng['execution_time']
    if 'cyclomatic_complexity' in X_eng.columns:
        X_eng['cpu_complexity_ratio'] = X_eng['cpu_percent_avg'] / (X_eng['cyclomatic_complexity'] + 1)

if 'cpu_percent_max' in X_eng.columns and 'cpu_percent_avg' in X_eng.columns:
    X_eng['cpu_variability'] = X_eng['cpu_percent_max'] - X_eng['cpu_percent_avg']

# === COMBINED ENERGY PROXY ===
print("   • Combined energy proxy...")

energy_proxy = 0
if 'execution_time' in X_eng.columns:
    energy_proxy += X_eng['execution_time'] * 50  # Time is major factor

if 'memory_peak_mb' in X_eng.columns:
    energy_proxy += X_eng['memory_peak_mb'] * 0.5  # Memory contributes

if 'cpu_percent_avg' in X_eng.columns:
    energy_proxy += X_eng['cpu_percent_avg'] * 2  # CPU contributes

if isinstance(energy_proxy, pd.Series):
    X_eng['combined_energy_proxy'] = energy_proxy

# === STATIC FEATURE ENGINEERING ===
print("   • Static feature polynomials...")

for col in ['total_lines', 'code_lines', 'cyclomatic_complexity', 'total_ast_nodes']:
    if col in X_eng.columns:
        X_eng[f'{col}_sqrt'] = np.sqrt(X_eng[col] + 1)
        X_eng[f'{col}_log'] = np.log1p(X_eng[col])
        X_eng[f'{col}_square'] = X_eng[col] ** 2

# Static interactions
if 'total_lines' in X_eng.columns and 'cyclomatic_complexity' in X_eng.columns:
    X_eng['lines_x_complexity'] = X_eng['total_lines'] * X_eng['cyclomatic_complexity']

if 'code_lines' in X_eng.columns and 'total_ast_nodes' in X_eng.columns:
    X_eng['code_x_ast'] = X_eng['code_lines'] * X_eng['total_ast_nodes']

# Ratios
if 'code_lines' in X_eng.columns and 'total_lines' in X_eng.columns:
    X_eng['code_ratio'] = X_eng['code_lines'] / (X_eng['total_lines'] + 1)

if 'comment_lines' in X_eng.columns and 'code_lines' in X_eng.columns:
    X_eng['comment_ratio_calc'] = X_eng['comment_lines'] / (X_eng['code_lines'] + 1)

print(f"✅ Engineered features: {X_eng.shape[1]} total")

# ============================================================================
# STEP 5: LOG TRANSFORM TARGET
# ============================================================================
y_log = np.log1p(y)
print(f"\n📈 Target (log): [{y_log.min():.2f}, {y_log.max():.2f}]")

# ============================================================================
# STEP 6: FEATURE SELECTION
# ============================================================================
print(f"\n🎯 Feature Selection...")

k_best = min(80, X_eng.shape[1])
selector = SelectKBest(score_func=mutual_info_regression, k=k_best)
selector.fit(X_eng, y_log)
X_selected = X_eng.loc[:, selector.get_support()]

print(f"✅ Selected {X_selected.shape[1]} features")

# ============================================================================
# STEP 7: NORMALIZE
# ============================================================================
qt = QuantileTransformer(
    n_quantiles=min(1000, X_selected.shape[0]),
    output_distribution='normal',
    random_state=42
)
X_transformed = pd.DataFrame(
    qt.fit_transform(X_selected),
    columns=X_selected.columns,
    index=X_selected.index
)

# ============================================================================
# STEP 8: STRATIFIED SPLIT
# ============================================================================
y_bins = pd.qcut(y_log, q=5, labels=False, duplicates='drop')
X_train, X_test, y_train, y_test = train_test_split(
    X_transformed, y_log, test_size=0.15, random_state=42,
    stratify=y_bins, shuffle=True
)

print(f"\n📊 Split: {X_train.shape[0]:,} train, {X_test.shape[0]:,} test")

# ============================================================================
# STEP 9: OPTIMIZED STACKING ENSEMBLE
# ============================================================================
print(f"\n🚀 Training Stacking Ensemble (Static + Dynamic Features)...")

# With dynamic features, we can use less regularization
base_models = [
    ('lgbm1', LGBMRegressor(
        n_estimators=1500, learning_rate=0.01, num_leaves=50,
        max_depth=10, min_child_samples=10, subsample=0.9,
        colsample_bytree=0.9, reg_alpha=0.1, reg_lambda=0.1,
        random_state=42, verbose=-1, n_jobs=-1
    )),
    ('lgbm2', LGBMRegressor(
        n_estimators=1200, learning_rate=0.015, num_leaves=35,
        max_depth=8, min_child_samples=15, subsample=0.85,
        colsample_bytree=0.85, reg_alpha=0.3, reg_lambda=0.3,
        random_state=43, verbose=-1, n_jobs=-1
    )),
    ('xgb1', XGBRegressor(
        n_estimators=1500, learning_rate=0.01, max_depth=10,
        min_child_weight=1, subsample=0.9, colsample_bytree=0.9,
        reg_alpha=0.1, reg_lambda=0.1, random_state=42,
        verbosity=0, n_jobs=-1
    )),
    ('xgb2', XGBRegressor(
        n_estimators=1200, learning_rate=0.015, max_depth=8,
        min_child_weight=2, subsample=0.85, colsample_bytree=0.85,
        reg_alpha=0.3, reg_lambda=0.3, random_state=43,
        verbosity=0, n_jobs=-1
    )),
    ('ridge', Ridge(alpha=1.0)),
    ('elastic', ElasticNet(alpha=0.001, l1_ratio=0.5, max_iter=5000))
]

meta_model = Ridge(alpha=1.0)

stacking_model = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5,
    n_jobs=-1
)

print("   Training ensemble...")
stacking_model.fit(X_train, y_train)

# ============================================================================
# STEP 10: EVALUATE
# ============================================================================
print(f"\n" + "=" * 80)
print("MODEL PERFORMANCE: STATIC + DYNAMIC (NO LEAKAGE)")
print("=" * 80)

y_train_pred_log = stacking_model.predict(X_train)
y_test_pred_log = stacking_model.predict(X_test)

r2_train_log = r2_score(y_train, y_train_pred_log)
r2_test_log = r2_score(y_test, y_test_pred_log)

print(f"\n📊 Log-Scale Performance:")
print(f"   Training R²: {r2_train_log:.6f}")
print(f"   Test R²:     {r2_test_log:.6f}")
print(f"   Overfitting Gap: {r2_train_log - r2_test_log:.6f}")

# Original scale
y_train_actual = np.expm1(y_train)
y_train_pred = np.expm1(y_train_pred_log)
y_test_actual = np.expm1(y_test)
y_test_pred = np.expm1(y_test_pred_log)

r2_train_original = r2_score(y_train_actual, y_train_pred)
r2_test_original = r2_score(y_test_actual, y_test_pred)

print(f"\n⚡ Original Joules Scale:")
print(f"   Training R²: {r2_train_original:.6f}")
print(f"   Test R²:     {r2_test_original:.6f}")

mae_test = mean_absolute_error(y_test_actual, y_test_pred)
rmse_test = np.sqrt(mean_squared_error(y_test_actual, y_test_pred))
mean_energy = y_test_actual.mean()

print(f"\n📏 Error Metrics:")
print(f"   Test MAE:  {mae_test:,.2f} J ({(mae_test/mean_energy)*100:.1f}% of mean)")
print(f"   Test RMSE: {rmse_test:,.2f} J")
print(f"   Mean Energy: {mean_energy:,.2f} J")

# ============================================================================
# STEP 11: CROSS-VALIDATION
# ============================================================================
print(f"\n🔄 5-Fold Cross-Validation:")
cv_scores = cross_val_score(
    stacking_model, X_transformed, y_log, cv=5,
    scoring='r2', n_jobs=-1
)
print(f"   CV R²: {[f'{s:.4f}' for s in cv_scores]}")
print(f"   Mean: {cv_scores.mean():.6f} (±{cv_scores.std():.6f})")

# ============================================================================
# STEP 12: FEATURE IMPORTANCE ANALYSIS
# ============================================================================
print(f"\n🎯 Top 20 Features (Static vs Dynamic):")
print("-" * 80)

# Get importance from LGBM
lgbm_model = base_models[0][1]
lgbm_model.fit(X_train, y_train)

feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': lgbm_model.feature_importances_
}).sort_values('importance', ascending=False)

for idx, row in feature_importance.head(20).iterrows():
    feat = row['feature']
    # Mark feature type
    if any(d in feat for d in ['execution_time', 'memory', 'cpu']):
        marker = "🔥 [DYNAMIC]"
    elif any(s in feat for s in ['time_per', 'memory_per', 'cpu_']):
        marker = "⚡ [HYBRID]"
    else:
        marker = "📊 [STATIC] "

    print(f"   {marker} {feat:40s} {row['importance']:8.1f}")

# ============================================================================
# STEP 13: DYNAMIC FEATURE CONTRIBUTION ANALYSIS
# ============================================================================
print(f"\n🔬 Dynamic Feature Impact Analysis:")

# Count dynamic features in top 20
top_20_features = feature_importance.head(20)['feature'].tolist()
dynamic_in_top = sum(1 for f in top_20_features
                     if any(d in f for d in ['execution_time', 'memory', 'cpu']))
hybrid_in_top = sum(1 for f in top_20_features
                    if any(h in f for h in ['time_per', 'memory_per', 'cpu_']))

print(f"   Dynamic features in Top 20: {dynamic_in_top}")
print(f"   Hybrid features in Top 20: {hybrid_in_top}")
print(f"   Static features in Top 20: {20 - dynamic_in_top - hybrid_in_top}")

# Total importance by type
total_importance = feature_importance['importance'].sum()
dynamic_importance = feature_importance[
    feature_importance['feature'].str.contains('execution_time|memory_|cpu_', regex=True)
]['importance'].sum()

print(f"\n   Dynamic feature importance: {dynamic_importance:,.0f} ({dynamic_importance/total_importance*100:.1f}%)")
print(f"   Static feature importance: {total_importance - dynamic_importance:,.0f} ({(1-dynamic_importance/total_importance)*100:.1f}%)")

# ============================================================================
# STEP 14: COMPARE WITH STATIC-ONLY
# ============================================================================
print(f"\n📊 Performance Comparison:")
print(f"   Static-only model (previous): R² ~ 0.53-0.65")
print(f"   Static + Dynamic (current):   R² = {r2_test_log:.4f}")
print(f"   Improvement: +{(r2_test_log - 0.59) * 100:.1f} percentage points")

# ============================================================================
# FINAL SUMMARY
# ============================================================================
print(f"\n" + "=" * 80)
print("🎉 FINAL RESULTS - STATIC + DYNAMIC FEATURES")
print("=" * 80)
print(f"✅ Samples: {X_train.shape[0]:,} train, {X_test.shape[0]:,} test")
print(f"✅ Features: {X_transformed.shape[1]} (static + dynamic, zero leakage)")
print(f"✅ Dynamic features used: {len(dynamic_used)}")
print(f"✅ Test R² (Log): {r2_test_log:.6f}")
print(f"✅ Test R² (Original): {r2_test_original:.6f}")
print(f"✅ CV R²: {cv_scores.mean():.6f} ±{cv_scores.std():.4f}")
print(f"✅ MAE: {mae_test:,.2f} J ({(mae_test/mean_energy)*100:.1f}%)")
print(f"✅ Overfitting: {r2_train_log - r2_test_log:.4f}")
print("=" * 80)

print(f"\n🔒 Leakage Prevention Summary:")
print(f"   ✅ Excluded: {len(LEAKAGE_FEATURES)} leakage features")
print(f"   ✅ Included: {len(dynamic_used)} clean dynamic features")
print(f"   ✅ No energy-derived features used")

if r2_test_log >= 0.9:
    print("\n🎊 EXCELLENT: R² ≥ 0.90 - Near-perfect prediction!")
elif r2_test_log >= 0.8:
    print("\n🚀 OUTSTANDING: R² ≥ 0.80 - Production-ready!")
elif r2_test_log >= 0.7:
    print("\n✅ STRONG: R² ≥ 0.70 - Very good performance!")
else:
    print(f"\n📊 Current: R² = {r2_test_log:.3f}")

print(f"\n💡 Key Insights:")
print(f"   • Dynamic features explain {dynamic_importance/total_importance*100:.0f}% of prediction power")
print(f"   • execution_time likely most important predictor")
print(f"   • Model shows upper bound of predictability")
print(f"   • Can be used for post-execution profiling/optimization")

STATIC + DYNAMIC FEATURES MODEL (ZERO LEAKAGE)

📊 Initial Dataset: 3,212 rows × 107 columns
✅ Valid samples: 626 rows

🔒 Building feature set with STRICT leakage prevention...

📋 Feature Breakdown:
   Static features: 86
   Dynamic features: 5
   Gray features: 7
   ✅ Total usable: 98
   ✅ No leakage detected!

🔥 Dynamic features included:
      • execution_time
      • cpu_percent_avg
      • cpu_percent_max
      • memory_usage_mb
      • memory_peak_mb

📐 Feature Matrix: 626 rows × 98 features

🧹 Data cleaning...
   Removed 8 constant features
✅ Clean data: 618 samples, 90 features

🔧 Feature Engineering...
   • Static × Dynamic interactions...
   • Combined energy proxy...
   • Static feature polynomials...
✅ Engineered features: 121 total

📈 Target (log): [1.59, 6.07]

🎯 Feature Selection...
✅ Selected 80 features

📊 Split: 525 train, 93 test

🚀 Training Stacking Ensemble (Static + Dynamic Features)...
   Training ensemble...

MODEL PERFORMANCE: STATIC + DYNAMIC (NO LEAKAGE)

📊 Lo

In [ ]:
#HYBRID
import pandas as pd
import numpy as np
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge, ElasticNet
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION
# ============================================================================
DATASET_PATH = '/content/ml_code_dataset.csv'

# ============================================================================
# FEATURE CATEGORIZATION - STRICT LEAKAGE PREVENTION
# ============================================================================

# TARGET (what we're predicting)
TARGET = 'energy_consumption_joules'

# LEAKAGE FEATURES - MUST BE REMOVED
# These are derived FROM the target or highly correlated outcomes
LEAKAGE_FEATURES = [
    'co2_emissions_g',           # Direct calculation from energy
    'energy_per_line',           # Derived: energy / lines
    'time_per_line',             # Derived: time / lines
    'co2_per_line',              # Derived: co2 / lines
    'energy_pkg_joules',         # Alternative energy measurement
    'energy_ram_joules',         # Alternative energy measurement
    'power_draw_watts',          # Direct energy measurement
]

# DYNAMIC FEATURES - CAN USE (measured during execution, but not derived from energy)
DYNAMIC_FEATURES = [
    'execution_time',            # Independent measurement
    'cpu_percent_avg',           # Independent measurement
    'cpu_percent_max',           # Independent measurement
    'memory_usage_mb',           # Independent measurement
    'memory_peak_mb',            # Independent measurement
]

# GRAY FEATURES - CAN USE
GRAY_FEATURES = [
    'executed_successfully', 'execution_error', 'exit_code',
    'syntax_valid', 'parse_error', 'syntax_error_line', 'syntax_error_type'
]

# STATIC FEATURES - CAN USE
STATIC_FEATURES = [
    'total_lines', 'code_lines', 'blank_lines', 'total_characters',
    'file_size_bytes', 'file_size_kb', 'avg_line_length', 'max_line_length',
    'min_line_length', 'std_line_length', 'comment_lines', 'single_quotes',
    'double_quotes', 'triple_quotes', 'total_quotes', 'parentheses_count',
    'brackets_count', 'braces_count', 'semicolon_count', 'colon_count',
    'unique_indentation_levels', 'avg_indentation', 'max_indentation',
    'std_indentation', 'keyword_def_count', 'keyword_class_count',
    'keyword_import_count', 'keyword_from_count', 'keyword_if_count',
    'keyword_else_count', 'keyword_elif_count', 'keyword_for_count',
    'keyword_while_count', 'keyword_try_count', 'keyword_except_count',
    'keyword_finally_count', 'keyword_with_count', 'keyword_return_count',
    'keyword_yield_count', 'keyword_lambda_count', 'keyword_async_count',
    'keyword_await_count', 'keyword_global_count', 'keyword_nonlocal_count',
    'functions_count', 'async_functions_count', 'classes_count',
    'imports_count', 'if_statements', 'for_loops', 'while_loops',
    'async_for_loops', 'try_blocks', 'with_statements', 'async_with_statements',
    'lambda_functions', 'list_comprehensions', 'dict_comprehensions',
    'set_comprehensions', 'generator_expressions', 'binary_operations',
    'unary_operations', 'boolean_operations', 'comparisons', 'decorators',
    'assignments', 'augmented_assignments', 'return_statements',
    'yield_statements', 'raise_statements', 'assert_statements',
    'cyclomatic_complexity', 'max_nesting_depth', 'unique_function_names',
    'unique_class_names', 'unique_imports', 'unique_variables',
    'total_ast_nodes', 'comment_ratio', 'control_flow_ratio',
    'function_ratio', 'class_ratio', 'code_density', 'complexity_per_line',
    'functions_per_line', 'avg_complexity_per_function'
]

# METADATA - EXCLUDE
METADATA_COLS = [
    'file_path', 'file_name', 'repo_name', 'repo_url', 'timestamp',
    'run_id', 'index', 'id', 'file_id', 'repository', 'author',
    'date_created', 'date_modified', 'file_index'
]

# ============================================================================
# STEP 1: LOAD DATA
# ============================================================================
print("=" * 80)
print("STATIC + DYNAMIC FEATURES MODEL (ZERO LEAKAGE)")
print("=" * 80)

df = pd.read_csv(DATASET_PATH)
print(f"\n📊 Initial Dataset: {df.shape[0]:,} rows × {df.shape[1]} columns")

if 'executed_successfully' in df.columns:
    df = df[df['executed_successfully'] == True].copy()

df = df[df[TARGET] > 0].copy()
df = df.dropna(subset=[TARGET])
print(f"✅ Valid samples: {df.shape[0]:,} rows")

# ============================================================================
# STEP 2: BUILD FEATURE SET (PREVENT LEAKAGE!)
# ============================================================================
print(f"\n🔒 Building feature set with STRICT leakage prevention...")

available_cols = df.columns.tolist()

# Collect all usable features
usable_features = []
for col in available_cols:
    # Include if it's static, dynamic, or gray
    if col in STATIC_FEATURES or col in DYNAMIC_FEATURES or col in GRAY_FEATURES:
        # Exclude if it's target, metadata, or leakage
        if col not in [TARGET] + METADATA_COLS + LEAKAGE_FEATURES:
            usable_features.append(col)

# Categorize features
static_used = [f for f in usable_features if f in STATIC_FEATURES]
dynamic_used = [f for f in usable_features if f in DYNAMIC_FEATURES]
gray_used = [f for f in usable_features if f in GRAY_FEATURES]

print(f"\n📋 Feature Breakdown:")
print(f"   Static features: {len(static_used)}")
print(f"   Dynamic features: {len(dynamic_used)}")
print(f"   Gray features: {len(gray_used)}")
print(f"   ✅ Total usable: {len(usable_features)}")

# Verify no leakage
leaked = [f for f in usable_features if f in LEAKAGE_FEATURES]
if leaked:
    print(f"   ❌ ERROR: Leaked features found: {leaked}")
    exit()
else:
    print(f"   ✅ No leakage detected!")

# Show dynamic features being used
if dynamic_used:
    print(f"\n🔥 Dynamic features included:")
    for feat in dynamic_used:
        print(f"      • {feat}")

X = df[usable_features].copy()
y = df[TARGET].copy()

print(f"\n📐 Feature Matrix: {X.shape[0]:,} rows × {X.shape[1]} features")

# ============================================================================
# STEP 3: MINIMAL CLEANING
# ============================================================================
print(f"\n🧹 Data cleaning...")

# Handle infinite/NaN
X.replace([np.inf, -np.inf], np.nan, inplace=True)
for col in X.columns:
    if X[col].dtype in ['float64', 'int64']:
        if X[col].isna().sum() > 0:
            X[col].fillna(X[col].median(), inplace=True)

# Remove constant features
constant_cols = [col for col in X.columns if X[col].nunique() <= 1]
if constant_cols:
    X = X.drop(columns=constant_cols)
    print(f"   Removed {len(constant_cols)} constant features")

# Very minimal outlier removal
q_low = y.quantile(0.005)
q_high = y.quantile(0.995)
mask = (y >= q_low) & (y <= q_high)
X = X[mask]
y = y[mask]

print(f"✅ Clean data: {X.shape[0]:,} samples, {X.shape[1]} features")

# ============================================================================
# STEP 4: FEATURE ENGINEERING
# ============================================================================
print(f"\n🔧 Feature Engineering...")

X_eng = X.copy()

# === INTERACTIONS BETWEEN STATIC AND DYNAMIC ===
print("   • Static × Dynamic interactions...")

# Execution time interactions
if 'execution_time' in X_eng.columns:
    if 'total_lines' in X_eng.columns:
        X_eng['time_per_line_calc'] = X_eng['execution_time'] / (X_eng['total_lines'] + 1)
    if 'cyclomatic_complexity' in X_eng.columns:
        X_eng['time_per_complexity'] = X_eng['execution_time'] / (X_eng['cyclomatic_complexity'] + 1)
    if 'for_loops' in X_eng.columns:
        X_eng['time_per_loop'] = X_eng['execution_time'] / (X_eng['for_loops'] + 1)

    # Time efficiency ratios
    X_eng['time_squared'] = X_eng['execution_time'] ** 2
    X_eng['time_log'] = np.log1p(X_eng['execution_time'])
    X_eng['time_sqrt'] = np.sqrt(X_eng['execution_time'])

# Memory interactions
if 'memory_peak_mb' in X_eng.columns:
    if 'total_lines' in X_eng.columns:
        X_eng['memory_per_line'] = X_eng['memory_peak_mb'] / (X_eng['total_lines'] + 1)
    if 'unique_variables' in X_eng.columns:
        X_eng['memory_per_var'] = X_eng['memory_peak_mb'] / (X_eng['unique_variables'] + 1)
    if 'classes_count' in X_eng.columns:
        X_eng['memory_per_class'] = X_eng['memory_peak_mb'] / (X_eng['classes_count'] + 1)

    X_eng['memory_squared'] = X_eng['memory_peak_mb'] ** 2
    X_eng['memory_log'] = np.log1p(X_eng['memory_peak_mb'])

# CPU interactions
if 'cpu_percent_avg' in X_eng.columns:
    if 'execution_time' in X_eng.columns:
        X_eng['cpu_time_product'] = X_eng['cpu_percent_avg'] * X_eng['execution_time']
    if 'cyclomatic_complexity' in X_eng.columns:
        X_eng['cpu_complexity_ratio'] = X_eng['cpu_percent_avg'] / (X_eng['cyclomatic_complexity'] + 1)

if 'cpu_percent_max' in X_eng.columns and 'cpu_percent_avg' in X_eng.columns:
    X_eng['cpu_variability'] = X_eng['cpu_percent_max'] - X_eng['cpu_percent_avg']

# === COMBINED ENERGY PROXY ===
print("   • Combined energy proxy...")

energy_proxy = 0
if 'execution_time' in X_eng.columns:
    energy_proxy += X_eng['execution_time'] * 50  # Time is major factor

if 'memory_peak_mb' in X_eng.columns:
    energy_proxy += X_eng['memory_peak_mb'] * 0.5  # Memory contributes

if 'cpu_percent_avg' in X_eng.columns:
    energy_proxy += X_eng['cpu_percent_avg'] * 2  # CPU contributes

if isinstance(energy_proxy, pd.Series):
    X_eng['combined_energy_proxy'] = energy_proxy

# === STATIC FEATURE ENGINEERING ===
print("   • Static feature polynomials...")

for col in ['total_lines', 'code_lines', 'cyclomatic_complexity', 'total_ast_nodes']:
    if col in X_eng.columns:
        X_eng[f'{col}_sqrt'] = np.sqrt(X_eng[col] + 1)
        X_eng[f'{col}_log'] = np.log1p(X_eng[col])
        X_eng[f'{col}_square'] = X_eng[col] ** 2

# Static interactions
if 'total_lines' in X_eng.columns and 'cyclomatic_complexity' in X_eng.columns:
    X_eng['lines_x_complexity'] = X_eng['total_lines'] * X_eng['cyclomatic_complexity']

if 'code_lines' in X_eng.columns and 'total_ast_nodes' in X_eng.columns:
    X_eng['code_x_ast'] = X_eng['code_lines'] * X_eng['total_ast_nodes']

# Ratios
if 'code_lines' in X_eng.columns and 'total_lines' in X_eng.columns:
    X_eng['code_ratio'] = X_eng['code_lines'] / (X_eng['total_lines'] + 1)

if 'comment_lines' in X_eng.columns and 'code_lines' in X_eng.columns:
    X_eng['comment_ratio_calc'] = X_eng['comment_lines'] / (X_eng['code_lines'] + 1)

print(f"✅ Engineered features: {X_eng.shape[1]} total")

# ============================================================================
# STEP 5: LOG TRANSFORM TARGET
# ============================================================================
y_log = np.log1p(y)
print(f"\n📈 Target (log): [{y_log.min():.2f}, {y_log.max():.2f}]")

# ============================================================================
# STEP 6: FEATURE SELECTION
# ============================================================================
print(f"\n🎯 Feature Selection...")

k_best = min(80, X_eng.shape[1])
selector = SelectKBest(score_func=mutual_info_regression, k=k_best)
selector.fit(X_eng, y_log)
X_selected = X_eng.loc[:, selector.get_support()]

print(f"✅ Selected {X_selected.shape[1]} features")

# ============================================================================
# STEP 7: NORMALIZE
# ============================================================================
qt = QuantileTransformer(
    n_quantiles=min(1000, X_selected.shape[0]),
    output_distribution='normal',
    random_state=42
)
X_transformed = pd.DataFrame(
    qt.fit_transform(X_selected),
    columns=X_selected.columns,
    index=X_selected.index
)

# ============================================================================
# STEP 8: STRATIFIED SPLIT
# ============================================================================
y_bins = pd.qcut(y_log, q=5, labels=False, duplicates='drop')
X_train, X_test, y_train, y_test = train_test_split(
    X_transformed, y_log, test_size=0.15, random_state=42,
    stratify=y_bins, shuffle=True
)

print(f"\n📊 Split: {X_train.shape[0]:,} train, {X_test.shape[0]:,} test")

# ============================================================================
# STEP 9: OPTIMIZED STACKING ENSEMBLE
# ============================================================================
print(f"\n🚀 Training Stacking Ensemble (Static + Dynamic Features)...")

# With dynamic features, we can use less regularization
base_models = [
    ('lgbm1', LGBMRegressor(
        n_estimators=1500, learning_rate=0.01, num_leaves=50,
        max_depth=10, min_child_samples=10, subsample=0.9,
        colsample_bytree=0.9, reg_alpha=0.1, reg_lambda=0.1,
        random_state=42, verbose=-1, n_jobs=-1
    )),
    ('lgbm2', LGBMRegressor(
        n_estimators=1200, learning_rate=0.015, num_leaves=35,
        max_depth=8, min_child_samples=15, subsample=0.85,
        colsample_bytree=0.85, reg_alpha=0.3, reg_lambda=0.3,
        random_state=43, verbose=-1, n_jobs=-1
    )),
    ('xgb1', XGBRegressor(
        n_estimators=1500, learning_rate=0.01, max_depth=10,
        min_child_weight=1, subsample=0.9, colsample_bytree=0.9,
        reg_alpha=0.1, reg_lambda=0.1, random_state=42,
        verbosity=0, n_jobs=-1
    )),
    ('xgb2', XGBRegressor(
        n_estimators=1200, learning_rate=0.015, max_depth=8,
        min_child_weight=2, subsample=0.85, colsample_bytree=0.85,
        reg_alpha=0.3, reg_lambda=0.3, random_state=43,
        verbosity=0, n_jobs=-1
    )),
    ('ridge', Ridge(alpha=1.0)),
    ('elastic', ElasticNet(alpha=0.001, l1_ratio=0.5, max_iter=5000))
]

meta_model = Ridge(alpha=1.0)

stacking_model = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5,
    n_jobs=-1
)

print("   Training ensemble...")
stacking_model.fit(X_train, y_train)

# ============================================================================
# STEP 10: EVALUATE
# ============================================================================
print(f"\n" + "=" * 80)
print("MODEL PERFORMANCE: STATIC + DYNAMIC (NO LEAKAGE)")
print("=" * 80)

y_train_pred_log = stacking_model.predict(X_train)
y_test_pred_log = stacking_model.predict(X_test)

r2_train_log = r2_score(y_train, y_train_pred_log)
r2_test_log = r2_score(y_test, y_test_pred_log)

print(f"\n📊 Log-Scale Performance:")
print(f"   Training R²: {r2_train_log:.6f}")
print(f"   Test R²:     {r2_test_log:.6f}")
print(f"   Overfitting Gap: {r2_train_log - r2_test_log:.6f}")

# Original scale
y_train_actual = np.expm1(y_train)
y_train_pred = np.expm1(y_train_pred_log)
y_test_actual = np.expm1(y_test)
y_test_pred = np.expm1(y_test_pred_log)

r2_train_original = r2_score(y_train_actual, y_train_pred)
r2_test_original = r2_score(y_test_actual, y_test_pred)

print(f"\n⚡ Original Joules Scale:")
print(f"   Training R²: {r2_train_original:.6f}")
print(f"   Test R²:     {r2_test_original:.6f}")

mae_test = mean_absolute_error(y_test_actual, y_test_pred)
rmse_test = np.sqrt(mean_squared_error(y_test_actual, y_test_pred))
mean_energy = y_test_actual.mean()

print(f"\n📏 Error Metrics:")
print(f"   Test MAE:  {mae_test:,.2f} J ({(mae_test/mean_energy)*100:.1f}% of mean)")
print(f"   Test RMSE: {rmse_test:,.2f} J")
print(f"   Mean Energy: {mean_energy:,.2f} J")

# ============================================================================
# STEP 11: CROSS-VALIDATION
# ============================================================================
print(f"\n🔄 5-Fold Cross-Validation:")
cv_scores = cross_val_score(
    stacking_model, X_transformed, y_log, cv=5,
    scoring='r2', n_jobs=-1
)
print(f"   CV R²: {[f'{s:.4f}' for s in cv_scores]}")
print(f"   Mean: {cv_scores.mean():.6f} (±{cv_scores.std():.6f})")

# ============================================================================
# STEP 12: FEATURE IMPORTANCE ANALYSIS
# ============================================================================
print(f"\n🎯 Top 20 Features (Static vs Dynamic):")
print("-" * 80)

# Get importance from LGBM
lgbm_model = base_models[0][1]
lgbm_model.fit(X_train, y_train)

feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': lgbm_model.feature_importances_
}).sort_values('importance', ascending=False)

for idx, row in feature_importance.head(20).iterrows():
    feat = row['feature']
    # Mark feature type
    if any(d in feat for d in ['execution_time', 'memory', 'cpu']):
        marker = "🔥 [DYNAMIC]"
    elif any(s in feat for s in ['time_per', 'memory_per', 'cpu_']):
        marker = "⚡ [HYBRID]"
    else:
        marker = "📊 [STATIC] "

    print(f"   {marker} {feat:40s} {row['importance']:8.1f}")

# ============================================================================
# STEP 13: DYNAMIC FEATURE CONTRIBUTION ANALYSIS
# ============================================================================
print(f"\n🔬 Dynamic Feature Impact Analysis:")

# Count dynamic features in top 20
top_20_features = feature_importance.head(20)['feature'].tolist()
dynamic_in_top = sum(1 for f in top_20_features
                     if any(d in f for d in ['execution_time', 'memory', 'cpu']))
hybrid_in_top = sum(1 for f in top_20_features
                    if any(h in f for h in ['time_per', 'memory_per', 'cpu_']))

print(f"   Dynamic features in Top 20: {dynamic_in_top}")
print(f"   Hybrid features in Top 20: {hybrid_in_top}")
print(f"   Static features in Top 20: {20 - dynamic_in_top - hybrid_in_top}")

# Total importance by type
total_importance = feature_importance['importance'].sum()
dynamic_importance = feature_importance[
    feature_importance['feature'].str.contains('execution_time|memory_|cpu_', regex=True)
]['importance'].sum()

print(f"\n   Dynamic feature importance: {dynamic_importance:,.0f} ({dynamic_importance/total_importance*100:.1f}%)")
print(f"   Static feature importance: {total_importance - dynamic_importance:,.0f} ({(1-dynamic_importance/total_importance)*100:.1f}%)")

# ============================================================================
# STEP 14: COMPARE WITH STATIC-ONLY
# ============================================================================
print(f"\n📊 Performance Comparison:")
print(f"   Static-only model (previous): R² ~ 0.53-0.65")
print(f"   Static + Dynamic (current):   R² = {r2_test_log:.4f}")
print(f"   Improvement: +{(r2_test_log - 0.59) * 100:.1f} percentage points")

# ============================================================================
# FINAL SUMMARY
# ============================================================================
print(f"\n" + "=" * 80)
print("🎉 FINAL RESULTS - STATIC + DYNAMIC FEATURES")
print("=" * 80)
print(f"✅ Samples: {X_train.shape[0]:,} train, {X_test.shape[0]:,} test")
print(f"✅ Features: {X_transformed.shape[1]} (static + dynamic, zero leakage)")
print(f"✅ Dynamic features used: {len(dynamic_used)}")
print(f"✅ Test R² (Log): {r2_test_log:.6f}")
print(f"✅ Test R² (Original): {r2_test_original:.6f}")
print(f"✅ CV R²: {cv_scores.mean():.6f} ±{cv_scores.std():.4f}")
print(f"✅ MAE: {mae_test:,.2f} J ({(mae_test/mean_energy)*100:.1f}%)")
print(f"✅ Overfitting: {r2_train_log - r2_test_log:.4f}")
print("=" * 80)

print(f"\n🔒 Leakage Prevention Summary:")
print(f"   ✅ Excluded: {len(LEAKAGE_FEATURES)} leakage features")
print(f"   ✅ Included: {len(dynamic_used)} clean dynamic features")
print(f"   ✅ No energy-derived features used")

if r2_test_log >= 0.9:
    print("\n🎊 EXCELLENT: R² ≥ 0.90 - Near-perfect prediction!")
elif r2_test_log >= 0.8:
    print("\n🚀 OUTSTANDING: R² ≥ 0.80 - Production-ready!")
elif r2_test_log >= 0.7:
    print("\n✅ STRONG: R² ≥ 0.70 - Very good performance!")
else:
    print(f"\n📊 Current: R² = {r2_test_log:.3f}")

print(f"\n💡 Key Insights:")
print(f"   • Dynamic features explain {dynamic_importance/total_importance*100:.0f}% of prediction power")
print(f"   • execution_time likely most important predictor")
print(f"   • Model shows upper bound of predictability")
print(f"   • Can be used for post-execution profiling/optimization")

STATIC + DYNAMIC FEATURES MODEL (ZERO LEAKAGE)

📊 Initial Dataset: 6,711 rows × 107 columns
✅ Valid samples: 1,567 rows

🔒 Building feature set with STRICT leakage prevention...

📋 Feature Breakdown:
   Static features: 86
   Dynamic features: 5
   Gray features: 7
   ✅ Total usable: 98
   ✅ No leakage detected!

🔥 Dynamic features included:
      • execution_time
      • cpu_percent_avg
      • cpu_percent_max
      • memory_usage_mb
      • memory_peak_mb

📐 Feature Matrix: 1,567 rows × 98 features

🧹 Data cleaning...
   Removed 6 constant features
✅ Clean data: 1,551 samples, 92 features

🔧 Feature Engineering...
   • Static × Dynamic interactions...
   • Combined energy proxy...
   • Static feature polynomials...
✅ Engineered features: 123 total

📈 Target (log): [1.46, 6.18]

🎯 Feature Selection...
✅ Selected 80 features

📊 Split: 1,318 train, 233 test

🚀 Training Stacking Ensemble (Static + Dynamic Features)...
   Training ensemble...

MODEL PERFORMANCE: STATIC + DYNAMIC (NO LEAKA

In [1]:
# ===============================================================
# FINAL FULL CODE — HYBRID (STATIC + PROXY) WITH RANDOM FOREST
# ===============================================================

import pandas as pd
import numpy as np
from sklearn.ensemble import StackingRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge, ElasticNet
import warnings
warnings.filterwarnings('ignore')

# ===============================================================
# CONFIG
# ===============================================================

DATASET_PATH = "/content/ml_code_dataset.csv"
TARGET = "energy_consumption_joules"

LEAKAGE_FEATURES = [
    'co2_emissions_g', 'energy_per_line', 'time_per_line',
    'co2_per_line', 'energy_pkg_joules', 'energy_ram_joules',
    'power_draw_watts'
]

DYNAMIC_FEATURES = [
    'execution_time', 'cpu_percent_avg', 'cpu_percent_max',
    'memory_usage_mb', 'memory_peak_mb'
]

GRAY_FEATURES = [
    'executed_successfully', 'execution_error',
    'exit_code', 'syntax_valid', 'parse_error'
]

STATIC_FEATURES = [
    'total_lines', 'code_lines', 'blank_lines', 'total_characters',
    'file_size_bytes', 'avg_line_length', 'max_line_length',
    'min_line_length', 'std_line_length', 'comment_lines',
    'single_quotes', 'double_quotes', 'triple_quotes', 'total_quotes',
    'parentheses_count', 'brackets_count', 'braces_count',
    'semicolon_count', 'colon_count', 'unique_indentation_levels',
    'avg_indentation', 'max_indentation', 'std_indentation',
    'keyword_def_count', 'keyword_class_count', 'keyword_import_count',
    'keyword_if_count', 'keyword_for_count', 'keyword_while_count',
    'keyword_try_count', 'functions_count', 'classes_count',
    'imports_count', 'if_statements', 'for_loops', 'while_loops',
    'try_blocks', 'with_statements', 'lambda_functions',
    'list_comprehensions', 'dict_comprehensions',
    'generator_expressions', 'binary_operations',
    'boolean_operations', 'comparisons', 'decorators', 'assignments',
    'return_statements', 'cyclomatic_complexity', 'max_nesting_depth',
    'total_ast_nodes', 'comment_ratio', 'control_flow_ratio',
    'function_ratio', 'class_ratio', 'code_density',
    'complexity_per_line'
]

METADATA_COLS = [
    'file_path', 'file_name', 'repo_name', 'repo_url',
    'timestamp', 'run_id', 'index', 'id', 'file_id',
    'repository', 'author', 'date_created', 'date_modified',
    'file_index'
]

# ===============================================================
# STEP 1 — LOAD
# ===============================================================

print("="*90)
print("FINAL HYBRID MODEL — STATIC + PSEUDO-DYNAMIC PROXIES + RANDOM FOREST")
print("="*90)

df = pd.read_csv(DATASET_PATH)
print(f"Loaded dataset: {df.shape[0]} rows, {df.shape[1]} columns")

if "executed_successfully" in df:
    df = df[df["executed_successfully"] == True]

df = df[df[TARGET] > 0].dropna(subset=[TARGET])
print(f"Valid samples after filtering: {df.shape[0]}")

# ===============================================================
# STEP 2 — SELECT FEATURES
# ===============================================================

usable_features = []
for col in df.columns:
    if col in STATIC_FEATURES or col in GRAY_FEATURES:
        if col not in LEAKAGE_FEATURES + METADATA_COLS + [TARGET]:
            usable_features.append(col)

X = df[usable_features].copy()
y = df[TARGET].copy()

print(f"Using {len(usable_features)} static + gray features")

# ===============================================================
# STEP 3 — CLEANING
# ===============================================================

X.replace([np.inf, -np.inf], np.nan, inplace=True)
for col in X.columns:
    if X[col].isna().sum() > 0:
        X[col].fillna(X[col].median(), inplace=True)

constant_cols = [c for c in X.columns if X[c].nunique() <= 1]
if constant_cols:
    X.drop(columns=constant_cols, inplace=True)

q_low, q_high = y.quantile(0.005), y.quantile(0.995)
mask = (y >= q_low) & (y <= q_high)
X, y = X.loc[mask], y.loc[mask]

print(f"Final cleaned dataset: {X.shape[0]} samples, {X.shape[1]} features")

# ===============================================================
# STEP 4 — PROXY FEATURES
# ===============================================================

X_eng = X.copy()

# Execution time proxy
tmps = []
if 'total_lines' in X: tmps.append(0.6 * X['total_lines'])
if 'total_ast_nodes' in X: tmps.append(0.3 * X['total_ast_nodes'])
if 'cyclomatic_complexity' in X: tmps.append(0.4 * X['cyclomatic_complexity'])

if tmps:
    t = sum(tmps)
    t = (t - t.min()) / (t.max() - t.min() + 1e-9)
    X_eng['est_execution_time_proxy'] = t * (X['total_lines'].median() + 1)

# Memory proxy
mc = []
if 'total_ast_nodes' in X: mc.append(0.6 * X['total_ast_nodes'])
if 'total_characters' in X: mc.append(0.2 * X['total_characters'])
if mc:
    m = sum(mc)
    m = (m - m.min()) / (m.max() - m.min() + 1e-9)
    X_eng['est_memory_proxy'] = m * (X['total_ast_nodes'].median() + 1)

# CPU proxy
cpu = []
if 'for_loops' in X: cpu.append(0.5 * X['for_loops'])
if 'max_nesting_depth' in X: cpu.append(0.4 * X['max_nesting_depth'])

if cpu:
    c = sum(cpu)
    c = (c - c.min()) / (c.max() - c.min() + 1e-9)
    X_eng['est_cpu_proxy'] = c * 100

# Proxy interactions
if 'est_execution_time_proxy' in X_eng:
    X_eng['proxy_time_square'] = X_eng['est_execution_time_proxy'] ** 2

if 'est_memory_proxy' in X_eng:
    X_eng['proxy_mem_log'] = np.log1p(X_eng['est_memory_proxy'])

if 'est_cpu_proxy' in X_eng and 'total_lines' in X_eng:
    X_eng['proxy_cpu_times_lines'] = X_eng['est_cpu_proxy'] * X_eng['total_lines']

# Polynomial features
for col in ['total_lines', 'code_lines', 'cyclomatic_complexity', 'total_ast_nodes']:
    if col in X_eng:
        X_eng[f"{col}_sqrt"] = np.sqrt(X_eng[col] + 1)
        X_eng[f"{col}_log"] = np.log1p(X_eng[col])
        X_eng[f"{col}_square"] = X_eng[col] ** 2

print(f"Engineered features: {X_eng.shape[1]}")

# ===============================================================
# STEP 5 — TARGET LOG
# ===============================================================

y_log = np.log1p(y)

# ===============================================================
# STEP 6 — SELECT BEST FEATURES
# ===============================================================

k = min(80, X_eng.shape[1])
selector = SelectKBest(mutual_info_regression, k=k)
selector.fit(X_eng.fillna(0), y_log)

X_sel = X_eng.loc[:, selector.get_support()].fillna(0)

print(f"Selected top {X_sel.shape[1]} features")

# ===============================================================
# STEP 7 — NORMALIZE
# ===============================================================

qt = QuantileTransformer(output_distribution="normal", random_state=42)
X_norm = pd.DataFrame(qt.fit_transform(X_sel), columns=X_sel.columns)

# ===============================================================
# STEP 8 — TRAIN/TEST SPLIT
# ===============================================================

bins = pd.qcut(y_log, q=5, labels=False, duplicates="drop")
X_train, X_test, y_train, y_test = train_test_split(
    X_norm, y_log, test_size=0.15, random_state=42, stratify=bins
)

# ===============================================================
# STEP 9 — STACKING ENSEMBLE (WITH RANDOM FOREST)
# ===============================================================

print("\nTraining Stacking Ensemble with RF...")

base_models = [
    ("lgbm", LGBMRegressor(
        n_estimators=1000, learning_rate=0.02,
        num_leaves=50, random_state=42, n_jobs=-1, verbose=-1
    )),
    ("xgb", XGBRegressor(
        n_estimators=800, learning_rate=0.02,
        max_depth=8, n_jobs=-1, verbosity=0, random_state=42
    )),
    ("rf", RandomForestRegressor(
        n_estimators=600, n_jobs=-1, random_state=42
    )),
    ("ridge", Ridge(alpha=1.0)),
    ("elastic", ElasticNet(alpha=0.001, l1_ratio=0.5, max_iter=5000))
]

meta_model = Ridge(alpha=1.0)

model = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5,
    n_jobs=-1
)

model.fit(X_train, y_train)

# ===============================================================
# STEP 10 — EVALUATE
# ===============================================================

y_pred_test_log = model.predict(X_test)
y_pred_test = np.expm1(y_pred_test_log)
y_actual_test = np.expm1(y_test)

r2_test = r2_score(y_test, y_pred_test_log)
mae = mean_absolute_error(y_actual_test, y_pred_test)
rmse = np.sqrt(mean_squared_error(y_actual_test, y_pred_test))

print("\nRESULTS")
print("="*70)
print(f"Test R² (log): {r2_test:.4f}")
print(f"MAE:           {mae:.2f} J")
print(f"RMSE:          {rmse:.2f} J")

# ===============================================================
# STEP 11 — CROSS VAL
# ===============================================================

cv_scores = cross_val_score(model, X_norm, y_log, cv=5, scoring="r2", n_jobs=-1)
print("\nCross-validation R²:", cv_scores)
print("Mean CV:", cv_scores.mean())

print("\n🎉 DONE — FINAL HYBRID MODEL WITH RF SUCCESSFULLY RUN.")


FINAL HYBRID MODEL — STATIC + PSEUDO-DYNAMIC PROXIES + RANDOM FOREST
Loaded dataset: 6711 rows, 107 columns
Valid samples after filtering: 1567
Using 62 static + gray features
Final cleaned dataset: 1551 samples, 58 features
Engineered features: 76
Selected top 76 features

Training Stacking Ensemble with RF...

RESULTS
Test R² (log): 0.5477
MAE:           32.91 J
RMSE:          67.85 J

Cross-validation R²: [ 0.59746617  0.49754899 -0.24223204  0.57130904  0.57001247]
Mean CV: 0.3988209266781783

🎉 DONE — FINAL HYBRID MODEL WITH RF SUCCESSFULLY RUN.


In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION
# ============================================================================
DATASET_PATH = '/content/ml_code_dataset.csv'

DYNAMIC_FEATURES = [
    'execution_time', 'cpu_percent_avg', 'cpu_percent_max',
    'memory_usage_mb', 'memory_peak_mb', 'co2_emissions_g',
    'energy_per_line', 'time_per_line', 'co2_per_line'
]

GRAY_FEATURES = [
    'executed_successfully', 'execution_error', 'exit_code',
    'syntax_valid', 'parse_error', 'syntax_error_line', 'syntax_error_type'
]

STATIC_FEATURES = [
    'total_lines', 'code_lines', 'blank_lines', 'total_characters',
    'file_size_bytes', 'file_size_kb', 'avg_line_length', 'max_line_length',
    'min_line_length', 'std_line_length', 'comment_lines', 'single_quotes',
    'double_quotes', 'triple_quotes', 'total_quotes', 'parentheses_count',
    'brackets_count', 'braces_count', 'semicolon_count', 'colon_count',
    'unique_indentation_levels', 'avg_indentation', 'max_indentation',
    'std_indentation', 'keyword_def_count', 'keyword_class_count',
    'keyword_import_count', 'keyword_from_count', 'keyword_if_count',
    'keyword_else_count', 'keyword_elif_count', 'keyword_for_count',
    'keyword_while_count', 'keyword_try_count', 'keyword_except_count',
    'keyword_finally_count', 'keyword_with_count', 'keyword_return_count',
    'keyword_yield_count', 'keyword_lambda_count', 'keyword_async_count',
    'keyword_await_count', 'keyword_global_count', 'keyword_nonlocal_count',
    'functions_count', 'async_functions_count', 'classes_count',
    'imports_count', 'if_statements', 'for_loops', 'while_loops',
    'async_for_loops', 'try_blocks', 'with_statements', 'async_with_statements',
    'lambda_functions', 'list_comprehensions', 'dict_comprehensions',
    'set_comprehensions', 'generator_expressions', 'binary_operations',
    'unary_operations', 'boolean_operations', 'comparisons', 'decorators',
    'assignments', 'augmented_assignments', 'return_statements',
    'yield_statements', 'raise_statements', 'assert_statements',
    'cyclomatic_complexity', 'max_nesting_depth', 'unique_function_names',
    'unique_class_names', 'unique_imports', 'unique_variables',
    'total_ast_nodes', 'comment_ratio', 'control_flow_ratio',
    'function_ratio', 'class_ratio', 'code_density', 'complexity_per_line',
    'functions_per_line', 'avg_complexity_per_function'
]

METADATA_COLS = [
    'file_path', 'file_name', 'repo_name', 'repo_url', 'timestamp',
    'run_id', 'index', 'id', 'file_id', 'repository', 'author',
    'date_created', 'date_modified', 'file_index'
]

TARGET = 'energy_consumption_joules'

# ============================================================================
# STEP 1: LOAD DATA
# ============================================================================
print("=" * 80)
print("PSEUDO-DYNAMIC FEATURES MODEL - BRIDGE THE GAP")
print("=" * 80)

df = pd.read_csv(DATASET_PATH)
print(f"\n📊 Initial Dataset: {df.shape[0]:,} rows × {df.shape[1]} columns")

if 'executed_successfully' in df.columns:
    df = df[df['executed_successfully'] == True].copy()

df = df[df[TARGET] > 0].copy()
df = df.dropna(subset=[TARGET])
print(f"✅ Valid samples: {df.shape[0]:,} rows")

# ============================================================================
# STEP 2: BUILD BASE FEATURES
# ============================================================================
available_cols = df.columns.tolist()
usable_features = [col for col in available_cols
                   if (col in STATIC_FEATURES or col in GRAY_FEATURES)
                   and col not in [TARGET] + METADATA_COLS + DYNAMIC_FEATURES]

X = df[usable_features].copy()
y = df[TARGET].copy()

# Clean
X.replace([np.inf, -np.inf], np.nan, inplace=True)
for col in X.columns:
    if X[col].dtype in ['float64', 'int64']:
        if X[col].isna().sum() > 0:
            X[col].fillna(X[col].median(), inplace=True)

constant_cols = [col for col in X.columns if X[col].nunique() <= 1]
if constant_cols:
    X = X.drop(columns=constant_cols)

# Minimal outlier removal
q_low = y.quantile(0.005)
q_high = y.quantile(0.995)
mask = (y >= q_low) & (y <= q_high)
X = X[mask]
y = y[mask]

print(f"📐 Base: {X.shape[0]:,} samples, {X.shape[1]} features")

# ============================================================================
# STEP 3: CREATE PSEUDO-DYNAMIC FEATURES
# ============================================================================
print(f"\n🎯 Creating Pseudo-Dynamic Features (Static → Runtime Proxies)...")

X_pseudo = X.copy()

# === ESTIMATED EXECUTION TIME PROXIES ===
print("   • Execution time proxies...")

# More loops = longer execution
if 'for_loops' in X_pseudo.columns and 'while_loops' in X_pseudo.columns:
    X_pseudo['estimated_loop_cost'] = (
        X_pseudo['for_loops'] * 10 +
        X_pseudo['while_loops'] * 15 +  # while loops often slower
        X_pseudo.get('async_for_loops', 0) * 12
    )

# Nested complexity = exponential time
if 'max_nesting_depth' in X_pseudo.columns and 'cyclomatic_complexity' in X_pseudo.columns:
    X_pseudo['estimated_time_complexity'] = (
        X_pseudo['max_nesting_depth'] ** 2 *
        X_pseudo['cyclomatic_complexity']
    )

# Lines × complexity = rough execution proxy
if 'code_lines' in X_pseudo.columns and 'cyclomatic_complexity' in X_pseudo.columns:
    X_pseudo['estimated_ops_count'] = (
        X_pseudo['code_lines'] *
        (X_pseudo['cyclomatic_complexity'] + 1) ** 1.5
    )

# Function calls (more functions = more overhead)
if 'functions_count' in X_pseudo.columns and 'total_lines' in X_pseudo.columns:
    X_pseudo['estimated_call_overhead'] = (
        X_pseudo['functions_count'] *
        np.log1p(X_pseudo['total_lines'])
    )

# === ESTIMATED MEMORY PROXIES ===
print("   • Memory usage proxies...")

# More variables = more memory
if 'unique_variables' in X_pseudo.columns:
    X_pseudo['estimated_var_memory'] = X_pseudo['unique_variables'] * np.log1p(X_pseudo['unique_variables'])

# Data structures = memory hogs
if 'list_comprehensions' in X_pseudo.columns:
    X_pseudo['estimated_list_memory'] = X_pseudo['list_comprehensions'] * 20
if 'dict_comprehensions' in X_pseudo.columns:
    X_pseudo['estimated_dict_memory'] = X_pseudo['dict_comprehensions'] * 30
if 'set_comprehensions' in X_pseudo.columns:
    X_pseudo['estimated_set_memory'] = X_pseudo['set_comprehensions'] * 25

# Total estimated data structure memory
X_pseudo['estimated_datastructure_cost'] = (
    X_pseudo.get('estimated_list_memory', 0) +
    X_pseudo.get('estimated_dict_memory', 0) +
    X_pseudo.get('estimated_set_memory', 0)
)

# Classes = objects = memory
if 'classes_count' in X_pseudo.columns and 'unique_variables' in X_pseudo.columns:
    X_pseudo['estimated_object_memory'] = (
        X_pseudo['classes_count'] *
        X_pseudo['unique_variables'] * 2
    )

# === ESTIMATED CPU INTENSITY ===
print("   • CPU intensity proxies...")

# Nested loops = CPU intensive
if 'for_loops' in X_pseudo.columns and 'max_nesting_depth' in X_pseudo.columns:
    X_pseudo['estimated_cpu_intensity'] = (
        X_pseudo['for_loops'] *
        (X_pseudo['max_nesting_depth'] ** 2)
    )

# Binary operations = computation
if 'binary_operations' in X_pseudo.columns:
    X_pseudo['estimated_computation_load'] = (
        X_pseudo['binary_operations'] *
        np.log1p(X_pseudo['binary_operations'])
    )

# Comparisons in loops = intensive
if 'comparisons' in X_pseudo.columns and 'for_loops' in X_pseudo.columns:
    X_pseudo['estimated_comparison_cost'] = (
        X_pseudo['comparisons'] *
        (X_pseudo['for_loops'] + 1)
    )

# === LIBRARY/IO PROXIES ===
print("   • I/O and library proxies...")

# More imports = potential for expensive operations
if 'imports_count' in X_pseudo.columns:
    X_pseudo['estimated_library_overhead'] = (
        X_pseudo['imports_count'] ** 1.5
    )

# File operations proxy (try/except suggests I/O)
if 'try_blocks' in X_pseudo.columns and 'with_statements' in X_pseudo.columns:
    X_pseudo['estimated_io_operations'] = (
        X_pseudo['try_blocks'] * 5 +
        X_pseudo['with_statements'] * 10
    )

# Async operations = I/O bound
if 'async_functions_count' in X_pseudo.columns:
    X_pseudo['estimated_async_cost'] = (
        X_pseudo['async_functions_count'] * 15
    )
if 'keyword_await_count' in X_pseudo.columns:
    X_pseudo['estimated_await_cost'] = (
        X_pseudo['keyword_await_count'] * 8
    )

# === ALGORITHMIC COMPLEXITY PROXIES ===
print("   • Algorithmic complexity proxies...")

# Nested loops = O(n²) or worse
if 'for_loops' in X_pseudo.columns and 'while_loops' in X_pseudo.columns:
    loops_total = X_pseudo['for_loops'] + X_pseudo['while_loops']
    X_pseudo['estimated_algo_complexity_quadratic'] = loops_total ** 2
    X_pseudo['estimated_algo_complexity_cubic'] = loops_total ** 3

# Recursion proxy (functions calling functions)
if 'functions_count' in X_pseudo.columns and 'keyword_return_count' in X_pseudo.columns:
    X_pseudo['estimated_recursion_depth'] = (
        X_pseudo['functions_count'] *
        X_pseudo['keyword_return_count'] / (X_pseudo['total_lines'] + 1)
    )

# List comprehensions in loops = nested iteration
if 'list_comprehensions' in X_pseudo.columns and 'for_loops' in X_pseudo.columns:
    X_pseudo['estimated_nested_iteration'] = (
        X_pseudo['list_comprehensions'] *
        X_pseudo['for_loops']
    )

# === CODE COMPLEXITY INDICATORS ===
print("   • Code complexity indicators...")

# Deep nesting = hard to optimize
if 'max_nesting_depth' in X_pseudo.columns:
    X_pseudo['nesting_penalty'] = X_pseudo['max_nesting_depth'] ** 3

# High cyclomatic complexity = many branches
if 'cyclomatic_complexity' in X_pseudo.columns:
    X_pseudo['branch_penalty'] = X_pseudo['cyclomatic_complexity'] ** 2

# Code density = packed logic
if 'code_density' in X_pseudo.columns:
    X_pseudo['density_penalty'] = X_pseudo['code_density'] ** 2

# === COMBINED ENERGY SCORE ===
print("   • Combined energy score...")

# Weighted combination of all proxies
X_pseudo['estimated_energy_score'] = (
    X_pseudo.get('estimated_ops_count', 0) * 0.3 +
    X_pseudo.get('estimated_loop_cost', 0) * 0.2 +
    X_pseudo.get('estimated_cpu_intensity', 0) * 0.15 +
    X_pseudo.get('estimated_var_memory', 0) * 0.1 +
    X_pseudo.get('estimated_io_operations', 0) * 0.1 +
    X_pseudo.get('estimated_library_overhead', 0) * 0.05 +
    X_pseudo.get('nesting_penalty', 0) * 0.05 +
    X_pseudo.get('branch_penalty', 0) * 0.05
)

print(f"\n✅ Pseudo-dynamic features created: {X_pseudo.shape[1]} total features")
print(f"   Original: {X.shape[1]}")
print(f"   New pseudo-dynamic: {X_pseudo.shape[1] - X.shape[1]}")

# ============================================================================
# STEP 4: STANDARD FEATURE ENGINEERING
# ============================================================================
print(f"\n🔧 Standard Feature Engineering...")

# Power transforms
for col in ['total_lines', 'code_lines', 'cyclomatic_complexity',
            'total_ast_nodes', 'estimated_energy_score']:
    if col in X_pseudo.columns:
        X_pseudo[f'{col}_sqrt'] = np.sqrt(X_pseudo[col] + 1)
        X_pseudo[f'{col}_log'] = np.log1p(X_pseudo[col])
        X_pseudo[f'{col}_square'] = X_pseudo[col] ** 2

# Key interactions
if 'estimated_energy_score' in X_pseudo.columns:
    if 'total_lines' in X_pseudo.columns:
        X_pseudo['energy_per_line_proxy'] = X_pseudo['estimated_energy_score'] / (X_pseudo['total_lines'] + 1)
    if 'cyclomatic_complexity' in X_pseudo.columns:
        X_pseudo['energy_x_complexity'] = X_pseudo['estimated_energy_score'] * X_pseudo['cyclomatic_complexity']

print(f"✅ Total features: {X_pseudo.shape[1]}")

# ============================================================================
# STEP 5: LOG TRANSFORM TARGET
# ============================================================================
y_log = np.log1p(y)

# ============================================================================
# STEP 6: FEATURE SELECTION
# ============================================================================
print(f"\n🎯 Feature Selection...")

k_best = min(70, X_pseudo.shape[1])
selector = SelectKBest(score_func=mutual_info_regression, k=k_best)
selector.fit(X_pseudo, y_log)
X_selected = X_pseudo.loc[:, selector.get_support()]

print(f"✅ Selected {X_selected.shape[1]} best features")

# ============================================================================
# STEP 7: NORMALIZE
# ============================================================================
qt = QuantileTransformer(n_quantiles=min(1000, X_selected.shape[0]),
                         output_distribution='normal', random_state=42)
X_transformed = pd.DataFrame(
    qt.fit_transform(X_selected),
    columns=X_selected.columns,
    index=X_selected.index
)

# ============================================================================
# STEP 8: STRATIFIED SPLIT
# ============================================================================
y_bins = pd.qcut(y_log, q=5, labels=False, duplicates='drop')
X_train, X_test, y_train, y_test = train_test_split(
    X_transformed, y_log, test_size=0.15, random_state=42,
    stratify=y_bins, shuffle=True
)

print(f"\n📊 Split: {X_train.shape[0]:,} train, {X_test.shape[0]:,} test")

# ============================================================================
# STEP 9: STACKING ENSEMBLE
# ============================================================================
print(f"\n🚀 Training Stacking Ensemble with Pseudo-Dynamic Features...")

base_models = [
    ('lgbm1', LGBMRegressor(
        n_estimators=1200, learning_rate=0.008, num_leaves=40,
        max_depth=8, min_child_samples=15, subsample=0.85,
        colsample_bytree=0.85, reg_alpha=0.3, reg_lambda=0.3,
        random_state=42, verbose=-1, n_jobs=-1
    )),
    ('lgbm2', LGBMRegressor(
        n_estimators=1000, learning_rate=0.012, num_leaves=25,
        max_depth=6, min_child_samples=25, subsample=0.75,
        colsample_bytree=0.75, reg_alpha=0.8, reg_lambda=0.8,
        random_state=43, verbose=-1, n_jobs=-1
    )),
    ('xgb1', XGBRegressor(
        n_estimators=1200, learning_rate=0.008, max_depth=8,
        min_child_weight=2, subsample=0.85, colsample_bytree=0.85,
        reg_alpha=0.3, reg_lambda=0.3, random_state=42,
        verbosity=0, n_jobs=-1
    )),
    ('xgb2', XGBRegressor(
        n_estimators=1000, learning_rate=0.012, max_depth=6,
        min_child_weight=4, subsample=0.75, colsample_bytree=0.75,
        reg_alpha=0.8, reg_lambda=0.8, random_state=43,
        verbosity=0, n_jobs=-1
    )),
    ('ridge', Ridge(alpha=5.0)),
    ('elastic', ElasticNet(alpha=0.005, l1_ratio=0.5, max_iter=5000))
]

meta_model = Ridge(alpha=3.0)

stacking_model = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5,
    n_jobs=-1
)

stacking_model.fit(X_train, y_train)

# ============================================================================
# STEP 10: EVALUATE
# ============================================================================
print(f"\n" + "=" * 80)
print("MODEL PERFORMANCE WITH PSEUDO-DYNAMIC FEATURES")
print("=" * 80)

y_train_pred_log = stacking_model.predict(X_train)
y_test_pred_log = stacking_model.predict(X_test)

r2_train_log = r2_score(y_train, y_train_pred_log)
r2_test_log = r2_score(y_test, y_test_pred_log)

print(f"\n📊 Log-Scale Performance:")
print(f"   Training R²: {r2_train_log:.6f}")
print(f"   Test R²:     {r2_test_log:.6f}")
print(f"   Gap: {r2_train_log - r2_test_log:.6f}")

y_train_actual = np.expm1(y_train)
y_train_pred = np.expm1(y_train_pred_log)
y_test_actual = np.expm1(y_test)
y_test_pred = np.expm1(y_test_pred_log)

r2_test_original = r2_score(y_test_actual, y_test_pred)
mae_test = mean_absolute_error(y_test_actual, y_test_pred)
rmse_test = np.sqrt(mean_squared_error(y_test_actual, y_test_pred))
mean_energy = y_test_actual.mean()

print(f"\n⚡ Original Joules Scale:")
print(f"   Test R²: {r2_test_original:.6f}")
print(f"   MAE: {mae_test:,.2f} J ({(mae_test/mean_energy)*100:.1f}%)")
print(f"   RMSE: {rmse_test:,.2f} J")

# ============================================================================
# STEP 11: CROSS-VALIDATION
# ============================================================================
print(f"\n🔄 5-Fold Cross-Validation:")
cv_scores = cross_val_score(
    stacking_model, X_transformed, y_log, cv=5,
    scoring='r2', n_jobs=-1
)
print(f"   CV R²: {[f'{s:.4f}' for s in cv_scores]}")
print(f"   Mean: {cv_scores.mean():.6f} (±{cv_scores.std():.6f})")

# ============================================================================
# STEP 12: FEATURE IMPORTANCE (TOP PSEUDO-DYNAMIC)
# ============================================================================
print(f"\n🎯 Top 15 Features (including pseudo-dynamic):")
print("-" * 70)

# Get feature importance from LGBM base model
lgbm_model = base_models[0][1]
lgbm_model.fit(X_train, y_train)

feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': lgbm_model.feature_importances_
}).sort_values('importance', ascending=False)

for idx, row in feature_importance.head(15).iterrows():
    marker = "🔥" if 'estimated' in row['feature'] or 'penalty' in row['feature'] else "  "
    print(f"   {marker} {row['feature']:45s} {row['importance']:8.1f}")

# ============================================================================
# FINAL SUMMARY
# ============================================================================
print(f"\n" + "=" * 80)
print("🎉 FINAL RESULTS - WITH PSEUDO-DYNAMIC FEATURES")
print("=" * 80)
print(f"✅ Samples: {X_train.shape[0]:,} train, {X_test.shape[0]:,} test")
print(f"✅ Features: {X_transformed.shape[1]} (with pseudo-dynamic proxies)")
print(f"✅ Test R² (Log): {r2_test_log:.6f}")
print(f"✅ Test R² (Original): {r2_test_original:.6f}")
print(f"✅ CV R²: {cv_scores.mean():.6f} ±{cv_scores.std():.4f}")
print(f"✅ MAE: {mae_test:,.2f} J ({(mae_test/mean_energy)*100:.1f}%)")
print("=" * 80)

if r2_test_log >= 0.8:
    print("\n🎊 BREAKTHROUGH: R² ≥ 0.80! 🎊")
elif r2_test_log >= 0.7:
    print(f"\n🚀 Excellent: R² = {r2_test_log:.3f} - Pseudo-dynamic features working!")
elif r2_test_log >= 0.6:
    print(f"\n✅ Strong: R² = {r2_test_log:.3f} - Significant improvement!")
else:
    print(f"\n📊 Current: R² = {r2_test_log:.3f}")

PSEUDO-DYNAMIC FEATURES MODEL - BRIDGE THE GAP

📊 Initial Dataset: 6,711 rows × 107 columns
✅ Valid samples: 1,567 rows
📐 Base: 1,551 samples, 87 features

🎯 Creating Pseudo-Dynamic Features (Static → Runtime Proxies)...
   • Execution time proxies...
   • Memory usage proxies...
   • CPU intensity proxies...
   • I/O and library proxies...
   • Algorithmic complexity proxies...
   • Code complexity indicators...
   • Combined energy score...

✅ Pseudo-dynamic features created: 112 total features
   Original: 87
   New pseudo-dynamic: 25

🔧 Standard Feature Engineering...
✅ Total features: 129

🎯 Feature Selection...
✅ Selected 70 best features

📊 Split: 1,318 train, 233 test

🚀 Training Stacking Ensemble with Pseudo-Dynamic Features...

MODEL PERFORMANCE WITH PSEUDO-DYNAMIC FEATURES

📊 Log-Scale Performance:
   Training R²: 0.890703
   Test R²:     0.527900
   Gap: 0.362803

⚡ Original Joules Scale:
   Test R²: 0.304367
   MAE: 34.23 J (63.0%)
   RMSE: 70.10 J

🔄 5-Fold Cross-Validati